TO DO:
1. ~Fix counties for unclaimed~
2. ~Fix counties for unidentified~
3. ~Add line to state_centroids with south pole coordinates and nonsense FIPS code~
4. ~Re-export state-level json (now that county fields have been updated within the databases and American Samoa has been removed)~
    * ~MAKE SURE TO MAP NAS to nonsense FIPS~
5. ~Add to county centroids:~
    * ~55 lines with south pole coordinates and nonsense county FIPS codes (state only_999)~
    * ~1 line with south pole coordinates and nonsense FIPS code (99)~
6. ~Format county data - state name, and then name and county FIPS code, to get GEOID~
7. ~Export county-level json~
    * ~any records with no county get pulled (to separate state FIPS with no county key and nonsense coordinates [south pole])~
    * ~make sure state name included as field with each database, not just FIPS code~
8. ~Re-export summary count JSON (as a few cases have been deleted)~
    * ~Address 21 NAs for Unclaimed?~
9. ~Clean null values for age and gender and sex and race / ethnicity~
10. ~Re-export state and county JSONS~
11. ~Make GeoJSON of state polygons~
12. ~Make GeoJSON of county polygons~
13. ~Make summary GeoJSON with county counts~
14. See how bad city data would be
    * likely need to make all city names .lower
15. re-check city nulls
16. Add new county name field to city_df based on GEOID (since county names spelled differently)
17. Re-run all steps (inc. summary county json and county polygons)

In [1]:
# import necessary packages
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import geojson

Notes:
1. Edits to Missing_04182020.csv prior to import
  * Virgin Islands (to United States Virgin Islands), 
  * Tennesse (to Tennessee), and 
  * Northern Mariana Islands (to Commonwealth of the Northern Mariana Islands)
  * Address all county nulls
2. Edits to Unclaimed_0418202.csv priort to import
  * Address all county nulls
3. Edits to Unidentified_04182020.csv prior to import
  * Virgin Islands (to United States Virgin Islands)
  * Address all county nulls
4. Edits to state_centroids_v2 prior to import
  * Add one row with south pole coordinates and nonsense FIPS code(99) - for cases w/ no city, county, or state
5. Edits to county_centroids_v2 prior to import
  * Add one row with south pole coordinates and nonsense FIPS code(99) - for cases w/ no city, county, or state
  * Add 55 rows with south pole coordinates and nonsense county FIPS codes(999) - for cases w/ no city or county

In [2]:
# Read in csvs
city_df = pd.read_csv('cities.csv')

In [3]:
state_centroids_df = pd.read_csv('state_centroids.csv')

In [4]:
# alternate centroids - with None option with FIPS 99
state_centroids_v2_df = pd.read_csv('state_centroids_v2.csv')

In [5]:
county_centroids_df = pd.read_csv('county_centroids.csv', encoding='Windows-1252')

In [6]:
# alternate centroids - with None options with count FIPS 999
county_centroids_v2_df = pd.read_csv('county_centroids_v2.csv', encoding='Windows-1252')

In [7]:
missing_df = pd.read_csv('Missing_04182020.csv')

In [8]:
unclaimed_df = pd.read_csv('Unclaimed_04182020.csv')

In [9]:
unidentified_df = pd.read_csv('Unidentified_04182020.csv')

In [10]:
# check dataframe
state_centroids_v2_df.head()

,OBJECTID,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,ORIG_FID,Lon_dd,Lat_dd
0,1,28,1779790,0400000US28,28,MS,Mississippi,0,1.220000e+11,3.926920e+09,1,-89.665425,32.751473
1,2,37,1027616,0400000US37,37,NC,North Carolina,0,1.260000e+11,1.346607e+10,2,-79.380052,35.542238
2,3,40,1102857,0400000US40,40,OK,Oklahoma,0,1.780000e+11,3.374588e+09,3,-97.508293,35.583486
3,4,51,1779803,0400000US51,51,VA,Virginia,0,1.020000e+11,8.528532e+09,4,-78.883285,37.512967
4,5,54,1779805,0400000US54,54,WV,West Virginia,0,6.226647e+10,4.890285e+08,5,-80.613731,38.642587


# Part 1 - Prep data by replacing null attribute values [NOT CITY, COUNTY, OR STATE - Those are addressed later]

### Clean Missing dataframe

##### Check for null values, and replace as needed

In [11]:
# check dataframe
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified
0,MP2341,5/8/1999,Reynolds,James,28,Opp,Autauga,Alabama,Male,Black / African American,2/14/2020
1,MP8522,1/15/2010,Walker,Jeremy,32,Prattville,Autauga,Alabama,Male,White / Caucasian,7/27/2015
2,MP50314,5/16/2018,Keszthelyi,Steve,79,Elberta,Baldwin,Alabama,Male,White / Caucasian,11/13/2019
3,MP43187,2/20/2018,johnson,abram,60,fairhope,Baldwin,Alabama,Male,White / Caucasian,2/13/2020
4,MP23019,11/5/2012,Shroyer,James,40,Fairhope,Baldwin,Alabama,Male,White / Caucasian,2/3/2016


In [12]:
## Check if any rows with null value for DLC [NOTE: Leave as NaN]
missing_attr_df = missing_df.loc[missing_df['DLC'].isnull()]
# len(missing_attr_df)
missing_attr_df

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified
14160,MP56286,NaN,DELAHAY,Glenora,NaN,York,York,Pennsylvania,Female,White / Caucasian,4/8/2020
16310,MP54713,NaN,Garcia,Ruben,NaN,Austin,Travis,Texas,Male,Hispanic / Latino,4/3/2020


In [13]:
## Check if any rows with null value for Missing Age
missing_attr_1_df = missing_df.loc[missing_df['Missing Age'].isnull()]
# len(missing_attr_1_df)
missing_attr_1_df

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified
3293,MP35843,7/20/1990,Renteria,Martin,NaN,Santa Ana,Orange,California,Male,Hispanic / Latino,11/21/2019
4756,MP59185,11/25/1970,Rindahl,Baby Boy,NaN,Wheat Ridge,Jefferson,Colorado,Male,White / Caucasian,1/27/2020
5480,MP38301,8/13/2014,Weedon,Nehemiah,NaN,Jacksonville,Duval,Florida,Male,Black / African American,4/13/2020
6208,MP38734,2/15/2016,Richardson,Baby,NaN,Orlando,Orange,Florida,Female,Black / African American,4/9/2020
6883,MP566,7/7/1999,Murphy,Sarah,NaN,Columbus,Muscogee,Georgia,Female,Black / African American,10/4/2011
7079,MP1284,4/21/2002,Martinez,Priscilla,NaN,Honokowai,Maui,Hawaii,Female,"White / Caucasian, Hispanic / Latino",3/5/2020
7085,MP1282,7/29/2005,Amaral,Robert,NaN,Lanai,Maui,Hawaii,Male,Other,3/5/2020
7826,MP17063,5/20/2003,Liendo,Fabian,NaN,Wapello,Louisa,Iowa,Male,"White / Caucasian, Hispanic / Latino",10/4/2017
12607,MP14033,4/1/1965,Franks,Elizabeth,NaN,Toledo,Lucas,Ohio,Female,White / Caucasian,6/1/2018
13678,MP20168,12/4/1974,Lloyd,Charles,NaN,Banks,Washington,Oregon,Male,White / Caucasian,3/2/2020


In [14]:
# Re-assign missing age values to be 'Unknown'
missing_df['Missing Age'] = missing_df['Missing Age'].fillna('Unknown')
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified
0,MP2341,5/8/1999,Reynolds,James,28,Opp,Autauga,Alabama,Male,Black / African American,2/14/2020
1,MP8522,1/15/2010,Walker,Jeremy,32,Prattville,Autauga,Alabama,Male,White / Caucasian,7/27/2015
2,MP50314,5/16/2018,Keszthelyi,Steve,79,Elberta,Baldwin,Alabama,Male,White / Caucasian,11/13/2019
3,MP43187,2/20/2018,johnson,abram,60,fairhope,Baldwin,Alabama,Male,White / Caucasian,2/13/2020
4,MP23019,11/5/2012,Shroyer,James,40,Fairhope,Baldwin,Alabama,Male,White / Caucasian,2/3/2016


In [15]:
## Double check re-assigned values
missing_age_test_df = missing_df.loc[missing_df['Missing Age']=='Unknown']
missing_age_test_df
# len(missing_age_test_df)

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified
3293,MP35843,7/20/1990,Renteria,Martin,Unknown,Santa Ana,Orange,California,Male,Hispanic / Latino,11/21/2019
4756,MP59185,11/25/1970,Rindahl,Baby Boy,Unknown,Wheat Ridge,Jefferson,Colorado,Male,White / Caucasian,1/27/2020
5480,MP38301,8/13/2014,Weedon,Nehemiah,Unknown,Jacksonville,Duval,Florida,Male,Black / African American,4/13/2020
6208,MP38734,2/15/2016,Richardson,Baby,Unknown,Orlando,Orange,Florida,Female,Black / African American,4/9/2020
6883,MP566,7/7/1999,Murphy,Sarah,Unknown,Columbus,Muscogee,Georgia,Female,Black / African American,10/4/2011
7079,MP1284,4/21/2002,Martinez,Priscilla,Unknown,Honokowai,Maui,Hawaii,Female,"White / Caucasian, Hispanic / Latino",3/5/2020
7085,MP1282,7/29/2005,Amaral,Robert,Unknown,Lanai,Maui,Hawaii,Male,Other,3/5/2020
7826,MP17063,5/20/2003,Liendo,Fabian,Unknown,Wapello,Louisa,Iowa,Male,"White / Caucasian, Hispanic / Latino",10/4/2017
12607,MP14033,4/1/1965,Franks,Elizabeth,Unknown,Toledo,Lucas,Ohio,Female,White / Caucasian,6/1/2018
13678,MP20168,12/4/1974,Lloyd,Charles,Unknown,Banks,Washington,Oregon,Male,White / Caucasian,3/2/2020


In [16]:
## Check if any rows with null value for Race / Ethnicity
missing_attr_2_df = missing_df.loc[missing_df['Race / Ethnicity'].isnull()]
# len(missing_attr_2_df)
missing_attr_2_df

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified
12167,MP55602,4/1/1955,Williams,James,27,Marion,McDowell,North Carolina,Male,NaN,8/1/2019


In [17]:
# Re-assign missing race / ethnicity values to be 'Uncertain'
missing_df['Race / Ethnicity'] = missing_df['Race / Ethnicity'].fillna('Uncertain')
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified
0,MP2341,5/8/1999,Reynolds,James,28,Opp,Autauga,Alabama,Male,Black / African American,2/14/2020
1,MP8522,1/15/2010,Walker,Jeremy,32,Prattville,Autauga,Alabama,Male,White / Caucasian,7/27/2015
2,MP50314,5/16/2018,Keszthelyi,Steve,79,Elberta,Baldwin,Alabama,Male,White / Caucasian,11/13/2019
3,MP43187,2/20/2018,johnson,abram,60,fairhope,Baldwin,Alabama,Male,White / Caucasian,2/13/2020
4,MP23019,11/5/2012,Shroyer,James,40,Fairhope,Baldwin,Alabama,Male,White / Caucasian,2/3/2016


In [18]:
## Double check re-assigned values
missing_race_test_df = missing_df.loc[missing_df['Race / Ethnicity']=='Uncertain']
missing_race_test_df
# len(missing_race_test_df)

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified
252,MP55684,11/22/1979,Hoffman,Ronald,37,Dutch Harbor,Aleutians West,Alaska,Male,Uncertain,4/14/2020
259,MP34147,7/24/2004,Flores-Mata,Valeriano,32,St. Paul,Aleutians West,Alaska,Male,Uncertain,4/14/2020
306,MP57385,8/27/1985,Garcia,James,26,Anchorage,Anchorage,Alaska,Male,Uncertain,4/14/2020
307,MP43344,8/17/1981,Hawk,Robert,34,Anchorage,Anchorage,Alaska,Male,Uncertain,2/5/2020
414,MP4342,5/1/2004,Keys,Avi,19,Fairbanks,Fairbanks North Star,Alaska,Male,Uncertain,4/18/2018
462,MP66815,7/18/1981,Riddell,Randall,26,Juneau,Juneau,Alaska,Male,Uncertain,4/16/2020
463,MP67097,7/18/1981,Mourant,Rob,23,Juneau,Juneau,Alaska,Male,Uncertain,4/16/2020
465,MP66843,9/6/1967,Johnston,Kenneth,42,Juneau,Juneau,Alaska,Male,Uncertain,4/14/2020
539,MP66592,11/27/1979,Pipkin,Jay,17,Soldotna,Kenai Peninsula,Alaska,Male,Uncertain,4/14/2020
602,MP29019,5/28/1972,Torsen,Melvin,23,Kodiak,Kodiak Island,Alaska,Male,Uncertain,4/14/2020


##### Force all names to lowercase then capitalize


In [19]:
# check dataframe
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified
0,MP2341,5/8/1999,Reynolds,James,28,Opp,Autauga,Alabama,Male,Black / African American,2/14/2020
1,MP8522,1/15/2010,Walker,Jeremy,32,Prattville,Autauga,Alabama,Male,White / Caucasian,7/27/2015
2,MP50314,5/16/2018,Keszthelyi,Steve,79,Elberta,Baldwin,Alabama,Male,White / Caucasian,11/13/2019
3,MP43187,2/20/2018,johnson,abram,60,fairhope,Baldwin,Alabama,Male,White / Caucasian,2/13/2020
4,MP23019,11/5/2012,Shroyer,James,40,Fairhope,Baldwin,Alabama,Male,White / Caucasian,2/3/2016


In [20]:
missing_df['Last Name'] = missing_df['Last Name'].str.lower()
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified
0,MP2341,5/8/1999,reynolds,James,28,Opp,Autauga,Alabama,Male,Black / African American,2/14/2020
1,MP8522,1/15/2010,walker,Jeremy,32,Prattville,Autauga,Alabama,Male,White / Caucasian,7/27/2015
2,MP50314,5/16/2018,keszthelyi,Steve,79,Elberta,Baldwin,Alabama,Male,White / Caucasian,11/13/2019
3,MP43187,2/20/2018,johnson,abram,60,fairhope,Baldwin,Alabama,Male,White / Caucasian,2/13/2020
4,MP23019,11/5/2012,shroyer,James,40,Fairhope,Baldwin,Alabama,Male,White / Caucasian,2/3/2016


In [21]:
missing_df['Last Name'] = missing_df['Last Name'].str.capitalize()
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified
0,MP2341,5/8/1999,Reynolds,James,28,Opp,Autauga,Alabama,Male,Black / African American,2/14/2020
1,MP8522,1/15/2010,Walker,Jeremy,32,Prattville,Autauga,Alabama,Male,White / Caucasian,7/27/2015
2,MP50314,5/16/2018,Keszthelyi,Steve,79,Elberta,Baldwin,Alabama,Male,White / Caucasian,11/13/2019
3,MP43187,2/20/2018,Johnson,abram,60,fairhope,Baldwin,Alabama,Male,White / Caucasian,2/13/2020
4,MP23019,11/5/2012,Shroyer,James,40,Fairhope,Baldwin,Alabama,Male,White / Caucasian,2/3/2016


In [22]:
missing_df['First Name'] = missing_df['First Name'].str.lower()
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified
0,MP2341,5/8/1999,Reynolds,james,28,Opp,Autauga,Alabama,Male,Black / African American,2/14/2020
1,MP8522,1/15/2010,Walker,jeremy,32,Prattville,Autauga,Alabama,Male,White / Caucasian,7/27/2015
2,MP50314,5/16/2018,Keszthelyi,steve,79,Elberta,Baldwin,Alabama,Male,White / Caucasian,11/13/2019
3,MP43187,2/20/2018,Johnson,abram,60,fairhope,Baldwin,Alabama,Male,White / Caucasian,2/13/2020
4,MP23019,11/5/2012,Shroyer,james,40,Fairhope,Baldwin,Alabama,Male,White / Caucasian,2/3/2016


In [23]:
missing_df['First Name'] = missing_df['First Name'].str.capitalize()
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified
0,MP2341,5/8/1999,Reynolds,James,28,Opp,Autauga,Alabama,Male,Black / African American,2/14/2020
1,MP8522,1/15/2010,Walker,Jeremy,32,Prattville,Autauga,Alabama,Male,White / Caucasian,7/27/2015
2,MP50314,5/16/2018,Keszthelyi,Steve,79,Elberta,Baldwin,Alabama,Male,White / Caucasian,11/13/2019
3,MP43187,2/20/2018,Johnson,Abram,60,fairhope,Baldwin,Alabama,Male,White / Caucasian,2/13/2020
4,MP23019,11/5/2012,Shroyer,James,40,Fairhope,Baldwin,Alabama,Male,White / Caucasian,2/3/2016


### Clean Unclaimed dataframe

##### Check for null values, and replace as needed

In [24]:
# check dataframe
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified
0,UCP68231,4/15/2020,Sanders,Stephen,Male,White / Caucasian,Joliet,Will,Illinois,4/16/2020
1,UCP68248,4/13/2020,Brookshire,Renee,Female,White / Caucasian,Joliet,Will,Illinois,4/16/2020
2,UCP68242,4/9/2020,Smith,Victoria,Female,White / Caucasian,Manhattan,New York,New York,4/16/2020
3,UCP68244,4/9/2020,Bellamy,Eldred,Male,Black / African American,Brooklyn,Kings,New York,4/16/2020
4,UCP68228,4/9/2020,Rodriguez,Elido,Male,Hispanic / Latino,Manhattan,New York,New York,4/15/2020


In [25]:
## Check if any rows with null value for DBF [NOTE: Leave as NaN]
unclaimed_attr_df = unclaimed_df.loc[unclaimed_df['DBF'].isnull()]
# len(unclaimed_attr_df)
unclaimed_attr_df

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified
7382,UCP66140,NaN,Beverly,John,NaN,NaN,NaN,Yakima,Washington,4/5/2020
7383,UCP66175,NaN,Bradford,Mary,NaN,NaN,NaN,Yakima,Washington,4/6/2020
7384,UCP66181,NaN,Neal,Terry,NaN,NaN,NaN,Yakima,Washington,4/6/2020
7385,UCP66087,NaN,Dick,Donald,NaN,NaN,NaN,Yakima,Washington,4/4/2020
7386,UCP66095,NaN,Coyle,Edgar,NaN,NaN,NaN,Yakima,Washington,4/4/2020
...,...,...,...,...,...,...,...,...,...,...
8330,UCP320,NaN,Baskin,Clyde,Male,Black / African American,Spokane Valley,Spokane,Washington,4/14/2020
8331,UCP341,NaN,Brotherton,Quilen,Male,NaN,NaN,Spokane,Washington,4/14/2020
8332,UCP326,NaN,Bell,Phillip,Male,NaN,NaN,Spokane,Washington,4/14/2020
8333,UCP345,NaN,Buonantony,Elena,Female,NaN,NaN,Spokane,Washington,4/14/2020


In [26]:
## Check if any rows with null value for last name
unclaimed_attr_1_df = unclaimed_df.loc[unclaimed_df['Last Name'].isnull()]
# len(unclaimed_attr_1_df)
unclaimed_attr_1_df

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified
7911,UCP62756,NaN,NaN,NaN,NaN,NaN,NaN,San Bernardino,California,12/19/2019
8265,UCP62750,NaN,NaN,NaN,NaN,NaN,NaN,San Bernardino,California,12/19/2019


In [27]:
# Re-assign missing last name values to be 'Unknown'
unclaimed_df['Last Name'] = unclaimed_df['Last Name'].fillna('Unknown')
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified
0,UCP68231,4/15/2020,Sanders,Stephen,Male,White / Caucasian,Joliet,Will,Illinois,4/16/2020
1,UCP68248,4/13/2020,Brookshire,Renee,Female,White / Caucasian,Joliet,Will,Illinois,4/16/2020
2,UCP68242,4/9/2020,Smith,Victoria,Female,White / Caucasian,Manhattan,New York,New York,4/16/2020
3,UCP68244,4/9/2020,Bellamy,Eldred,Male,Black / African American,Brooklyn,Kings,New York,4/16/2020
4,UCP68228,4/9/2020,Rodriguez,Elido,Male,Hispanic / Latino,Manhattan,New York,New York,4/15/2020


In [28]:
## Double check re-assigned values
unclaimed_lname_test_df = unclaimed_df.loc[unclaimed_df['Last Name']=='Unknown']
unclaimed_lname_test_df
# len(unclaimed_lname_test_df)

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified
7911,UCP62756,NaN,Unknown,NaN,NaN,NaN,NaN,San Bernardino,California,12/19/2019
8265,UCP62750,NaN,Unknown,NaN,NaN,NaN,NaN,San Bernardino,California,12/19/2019


In [29]:
## Check if any rows with null value for first name
unclaimed_attr_2_df = unclaimed_df.loc[unclaimed_df['First Name'].isnull()]
# len(unclaimed_attr_2_df)
unclaimed_attr_2_df

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified
7911,UCP62756,NaN,Unknown,NaN,NaN,NaN,NaN,San Bernardino,California,12/19/2019
8168,UCP66063,NaN,Goldstein,NaN,NaN,NaN,NaN,Yakima,Washington,3/26/2020
8265,UCP62750,NaN,Unknown,NaN,NaN,NaN,NaN,San Bernardino,California,12/19/2019


In [30]:
# Re-assign missing first name values to be 'Unknown'
unclaimed_df['First Name'] = unclaimed_df['First Name'].fillna('Unknown')
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified
0,UCP68231,4/15/2020,Sanders,Stephen,Male,White / Caucasian,Joliet,Will,Illinois,4/16/2020
1,UCP68248,4/13/2020,Brookshire,Renee,Female,White / Caucasian,Joliet,Will,Illinois,4/16/2020
2,UCP68242,4/9/2020,Smith,Victoria,Female,White / Caucasian,Manhattan,New York,New York,4/16/2020
3,UCP68244,4/9/2020,Bellamy,Eldred,Male,Black / African American,Brooklyn,Kings,New York,4/16/2020
4,UCP68228,4/9/2020,Rodriguez,Elido,Male,Hispanic / Latino,Manhattan,New York,New York,4/15/2020


In [31]:
## Double check re-assigned values
unclaimed_fname_test_df = unclaimed_df.loc[unclaimed_df['First Name']=='Unknown']
unclaimed_fname_test_df
# len(unclaimed_fname_test_df)

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified
7911,UCP62756,NaN,Unknown,Unknown,NaN,NaN,NaN,San Bernardino,California,12/19/2019
8168,UCP66063,NaN,Goldstein,Unknown,NaN,NaN,NaN,Yakima,Washington,3/26/2020
8265,UCP62750,NaN,Unknown,Unknown,NaN,NaN,NaN,San Bernardino,California,12/19/2019


In [32]:
## Check if any rows with null value for sex
unclaimed_attr_3_df = unclaimed_df.loc[unclaimed_df['Sex'].isnull()]
# len(unclaimed_attr_3_df)
unclaimed_attr_3_df

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified
4134,UCP63503,1/22/2016,Vanderbeck,Robert,NaN,NaN,NaN,San Bernardino,California,12/19/2019
4160,UCP63509,1/10/2016,Johnson,James,NaN,NaN,NaN,San Bernardino,California,12/19/2019
4178,UCP63504,1/3/2016,McKenzie,Rexford,NaN,NaN,NaN,San Bernardino,California,12/19/2019
4181,UCP63505,12/31/2015,Schafer,Julia,NaN,NaN,NaN,San Bernardino,California,12/19/2019
4185,UCP63497,12/29/2015,Waldren,Hayward,NaN,NaN,NaN,San Bernardino,California,12/19/2019
...,...,...,...,...,...,...,...,...,...,...
8288,UCP62761,NaN,Sepulveda,Raymond,NaN,NaN,NaN,San Bernardino,California,12/19/2019
8289,UCP62762,NaN,Cruz,Sergio,NaN,NaN,NaN,San Bernardino,California,12/19/2019
8290,UCP62738,NaN,Hartman,William,NaN,NaN,NaN,San Bernardino,California,12/19/2019
8314,UCP1027,NaN,Miah,Mashuk,NaN,NaN,NaN,Wayne,Michigan,3/19/2020


In [33]:
# Re-assign missing sex values to be 'Unknown'
unclaimed_df['Sex'] = unclaimed_df['Sex'].fillna('Unknown')
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified
0,UCP68231,4/15/2020,Sanders,Stephen,Male,White / Caucasian,Joliet,Will,Illinois,4/16/2020
1,UCP68248,4/13/2020,Brookshire,Renee,Female,White / Caucasian,Joliet,Will,Illinois,4/16/2020
2,UCP68242,4/9/2020,Smith,Victoria,Female,White / Caucasian,Manhattan,New York,New York,4/16/2020
3,UCP68244,4/9/2020,Bellamy,Eldred,Male,Black / African American,Brooklyn,Kings,New York,4/16/2020
4,UCP68228,4/9/2020,Rodriguez,Elido,Male,Hispanic / Latino,Manhattan,New York,New York,4/15/2020


In [34]:
## Double check re-assigned values
unclaimed_sex_test_df = unclaimed_df.loc[unclaimed_df['Sex']=='Unknown']
unclaimed_sex_test_df
# len(unclaimed_sex_test_df)

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified
4134,UCP63503,1/22/2016,Vanderbeck,Robert,Unknown,NaN,NaN,San Bernardino,California,12/19/2019
4160,UCP63509,1/10/2016,Johnson,James,Unknown,NaN,NaN,San Bernardino,California,12/19/2019
4178,UCP63504,1/3/2016,McKenzie,Rexford,Unknown,NaN,NaN,San Bernardino,California,12/19/2019
4181,UCP63505,12/31/2015,Schafer,Julia,Unknown,NaN,NaN,San Bernardino,California,12/19/2019
4185,UCP63497,12/29/2015,Waldren,Hayward,Unknown,NaN,NaN,San Bernardino,California,12/19/2019
...,...,...,...,...,...,...,...,...,...,...
8288,UCP62761,NaN,Sepulveda,Raymond,Unknown,NaN,NaN,San Bernardino,California,12/19/2019
8289,UCP62762,NaN,Cruz,Sergio,Unknown,NaN,NaN,San Bernardino,California,12/19/2019
8290,UCP62738,NaN,Hartman,William,Unknown,NaN,NaN,San Bernardino,California,12/19/2019
8314,UCP1027,NaN,Miah,Mashuk,Unknown,NaN,NaN,Wayne,Michigan,3/19/2020


In [35]:
## Check if any rows with null value for race / ethnicity
unclaimed_attr_4_df = unclaimed_df.loc[unclaimed_df['Race / Ethnicity'].isnull()]
# len(unclaimed_attr_4_df)
unclaimed_attr_4_df

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified
2182,UCP5385,2/18/2018,Tharpe,Ronald,Male,NaN,lakewood,Pierce,Washington,5/5/2018
2240,UCP5361,2/1/2018,Weigand,Michael,Male,NaN,Tacoma,Pierce,Washington,5/5/2018
2241,UCP5384,2/1/2018,Weigand,Michael,Male,NaN,Tacoma,Pierce,Washington,5/5/2018
2245,UCP5398,1/31/2018,Campbell,Mollie,Female,NaN,Puyallup,Pierce,Washington,5/8/2018
2268,UCP5363,1/22/2018,Claxton,Sueann,Female,NaN,Tacoma,Pierce,Washington,5/5/2018
...,...,...,...,...,...,...,...,...,...,...
8329,UCP1040,NaN,Snell,Gerald,Male,NaN,NaN,Wayne,Michigan,3/19/2020
8331,UCP341,NaN,Brotherton,Quilen,Male,NaN,NaN,Spokane,Washington,4/14/2020
8332,UCP326,NaN,Bell,Phillip,Male,NaN,NaN,Spokane,Washington,4/14/2020
8333,UCP345,NaN,Buonantony,Elena,Female,NaN,NaN,Spokane,Washington,4/14/2020


In [36]:
# Re-assign missing race / ethnicity values to be 'Uncertain'
unclaimed_df['Race / Ethnicity'] = unclaimed_df['Race / Ethnicity'].fillna('Uncertain')
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified
0,UCP68231,4/15/2020,Sanders,Stephen,Male,White / Caucasian,Joliet,Will,Illinois,4/16/2020
1,UCP68248,4/13/2020,Brookshire,Renee,Female,White / Caucasian,Joliet,Will,Illinois,4/16/2020
2,UCP68242,4/9/2020,Smith,Victoria,Female,White / Caucasian,Manhattan,New York,New York,4/16/2020
3,UCP68244,4/9/2020,Bellamy,Eldred,Male,Black / African American,Brooklyn,Kings,New York,4/16/2020
4,UCP68228,4/9/2020,Rodriguez,Elido,Male,Hispanic / Latino,Manhattan,New York,New York,4/15/2020


In [37]:
## Double check re-assigned values
unclaimed_race_test_df = unclaimed_df.loc[unclaimed_df['Race / Ethnicity']=='Uncertain']
unclaimed_race_test_df
# len(unclaimed_race_test_df)

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified
66,UCP68060,3/21/2020,Moren,Carmen,Female,Uncertain,Bronx,Bronx,New York,4/11/2020
80,UCP67791,3/18/2020,Bekeova,Ilona,Female,Uncertain,Manhattan,New York,New York,4/3/2020
98,UCP68150,3/12/2020,Bell,Thomas,Male,Uncertain,Puyallup,Pierce,Washington,4/13/2020
99,UCP67863,3/11/2020,Olave,Rigoberto,Male,Uncertain,Queens,Queens,New York,4/5/2020
108,UCP68044,3/9/2020,Alarcon,Jose,Male,Uncertain,Bronx,Bronx,New York,4/10/2020
...,...,...,...,...,...,...,...,...,...,...
8329,UCP1040,NaN,Snell,Gerald,Male,Uncertain,NaN,Wayne,Michigan,3/19/2020
8331,UCP341,NaN,Brotherton,Quilen,Male,Uncertain,NaN,Spokane,Washington,4/14/2020
8332,UCP326,NaN,Bell,Phillip,Male,Uncertain,NaN,Spokane,Washington,4/14/2020
8333,UCP345,NaN,Buonantony,Elena,Female,Uncertain,NaN,Spokane,Washington,4/14/2020


##### Force all names to lowercase, then capitalize

In [38]:
# check dataframe
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified
0,UCP68231,4/15/2020,Sanders,Stephen,Male,White / Caucasian,Joliet,Will,Illinois,4/16/2020
1,UCP68248,4/13/2020,Brookshire,Renee,Female,White / Caucasian,Joliet,Will,Illinois,4/16/2020
2,UCP68242,4/9/2020,Smith,Victoria,Female,White / Caucasian,Manhattan,New York,New York,4/16/2020
3,UCP68244,4/9/2020,Bellamy,Eldred,Male,Black / African American,Brooklyn,Kings,New York,4/16/2020
4,UCP68228,4/9/2020,Rodriguez,Elido,Male,Hispanic / Latino,Manhattan,New York,New York,4/15/2020


In [39]:
unclaimed_df['Last Name'] = unclaimed_df['Last Name'].str.lower()
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified
0,UCP68231,4/15/2020,sanders,Stephen,Male,White / Caucasian,Joliet,Will,Illinois,4/16/2020
1,UCP68248,4/13/2020,brookshire,Renee,Female,White / Caucasian,Joliet,Will,Illinois,4/16/2020
2,UCP68242,4/9/2020,smith,Victoria,Female,White / Caucasian,Manhattan,New York,New York,4/16/2020
3,UCP68244,4/9/2020,bellamy,Eldred,Male,Black / African American,Brooklyn,Kings,New York,4/16/2020
4,UCP68228,4/9/2020,rodriguez,Elido,Male,Hispanic / Latino,Manhattan,New York,New York,4/15/2020


In [40]:
unclaimed_df['Last Name'] = unclaimed_df['Last Name'].str.capitalize()
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified
0,UCP68231,4/15/2020,Sanders,Stephen,Male,White / Caucasian,Joliet,Will,Illinois,4/16/2020
1,UCP68248,4/13/2020,Brookshire,Renee,Female,White / Caucasian,Joliet,Will,Illinois,4/16/2020
2,UCP68242,4/9/2020,Smith,Victoria,Female,White / Caucasian,Manhattan,New York,New York,4/16/2020
3,UCP68244,4/9/2020,Bellamy,Eldred,Male,Black / African American,Brooklyn,Kings,New York,4/16/2020
4,UCP68228,4/9/2020,Rodriguez,Elido,Male,Hispanic / Latino,Manhattan,New York,New York,4/15/2020


In [41]:
unclaimed_df['First Name'] = unclaimed_df['First Name'].str.lower()
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified
0,UCP68231,4/15/2020,Sanders,stephen,Male,White / Caucasian,Joliet,Will,Illinois,4/16/2020
1,UCP68248,4/13/2020,Brookshire,renee,Female,White / Caucasian,Joliet,Will,Illinois,4/16/2020
2,UCP68242,4/9/2020,Smith,victoria,Female,White / Caucasian,Manhattan,New York,New York,4/16/2020
3,UCP68244,4/9/2020,Bellamy,eldred,Male,Black / African American,Brooklyn,Kings,New York,4/16/2020
4,UCP68228,4/9/2020,Rodriguez,elido,Male,Hispanic / Latino,Manhattan,New York,New York,4/15/2020


In [42]:
unclaimed_df['First Name'] = unclaimed_df['First Name'].str.capitalize()
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified
0,UCP68231,4/15/2020,Sanders,Stephen,Male,White / Caucasian,Joliet,Will,Illinois,4/16/2020
1,UCP68248,4/13/2020,Brookshire,Renee,Female,White / Caucasian,Joliet,Will,Illinois,4/16/2020
2,UCP68242,4/9/2020,Smith,Victoria,Female,White / Caucasian,Manhattan,New York,New York,4/16/2020
3,UCP68244,4/9/2020,Bellamy,Eldred,Male,Black / African American,Brooklyn,Kings,New York,4/16/2020
4,UCP68228,4/9/2020,Rodriguez,Elido,Male,Hispanic / Latino,Manhattan,New York,New York,4/15/2020


### Clean Unidentified dataframe

##### Check for null values, and replace as needed

In [43]:
# check dataframe
unidentified_df.head()

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified
0,UP51903,3/30/1986,NaN,NaN,Marbury,Autauga,Alabama,Female,White / Caucasian,1/28/2020
1,UP5000,9/29/2007,40.0,60.0,Prattville,Autauga,Alabama,Female,Black / African American,8/12/2019
2,UP6519,1/6/2006,45.0,59.0,Elberta,Baldwin,Alabama,Male,White / Caucasian,10/24/2018
3,UP55409,3/26/2000,NaN,NaN,NaN,Baldwin,Alabama,Male,"White / Caucasian, Asian, American Indian / Al...",2/13/2020
4,UP13483,3/27/1961,14.0,17.0,NaN,Bibb,Alabama,Male,White / Caucasian,3/25/2019


In [44]:
## Check if any rows with null value for DBF [NOTE: Leave as NaN]
unidentified_attr_df = unidentified_df.loc[unidentified_df['DBF'].isnull()]
# len(unidentified_attr_df)
unidentified_attr_df

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified
1621,UP54136,NaN,NaN,NaN,NaN,Pima,Arizona,NaN,Uncertain,1/16/2020
3718,UP17837,NaN,65.0,70.0,NaN,Orange,California,Male,White / Caucasian,11/1/2018


In [45]:
## Check if any rows with null value for Age From
unidentified_attr_1_df = unidentified_df.loc[unidentified_df['Age From'].isnull()]
# len(unidentified_attr_1_df)
unidentified_attr_1_df

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified
0,UP51903,3/30/1986,NaN,NaN,Marbury,Autauga,Alabama,Female,White / Caucasian,1/28/2020
3,UP55409,3/26/2000,NaN,NaN,NaN,Baldwin,Alabama,Male,"White / Caucasian, Asian, American Indian / Al...",2/13/2020
34,UP51245,11/29/2017,NaN,NaN,Huntsville,Madison,Alabama,Male,Uncertain,12/27/2019
35,UP11867,12/8/2013,NaN,NaN,Huntsville,Madison,Alabama,Male,White / Caucasian,11/9/2018
36,UP51243,11/29/2017,NaN,NaN,Huntsville,Madison,Alabama,Female,Uncertain,12/27/2019
...,...,...,...,...,...,...,...,...,...,...
13206,UP52045,6/14/2018,NaN,NaN,Cyclone,Wyoming,West Virginia,Male,Uncertain,1/15/2020
13216,UP13695,11/9/1989,NaN,NaN,Paris,Kenosha,Wisconsin,Male,White / Caucasian,4/11/2017
13240,UP11611,6/5/2008,NaN,NaN,Baraboo,Sauk,Wisconsin,Male,Uncertain,11/22/2013
13248,UP7631,7/10/1973,NaN,NaN,Waukesha,Waukesha,Wisconsin,Male,White / Caucasian,11/9/2013


In [46]:
# Re-assign missing age from values to be 0
unidentified_df['Age From'] = unidentified_df['Age From'].fillna(0)
unidentified_df.head()

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified
0,UP51903,3/30/1986,0.0,NaN,Marbury,Autauga,Alabama,Female,White / Caucasian,1/28/2020
1,UP5000,9/29/2007,40.0,60.0,Prattville,Autauga,Alabama,Female,Black / African American,8/12/2019
2,UP6519,1/6/2006,45.0,59.0,Elberta,Baldwin,Alabama,Male,White / Caucasian,10/24/2018
3,UP55409,3/26/2000,0.0,NaN,NaN,Baldwin,Alabama,Male,"White / Caucasian, Asian, American Indian / Al...",2/13/2020
4,UP13483,3/27/1961,14.0,17.0,NaN,Bibb,Alabama,Male,White / Caucasian,3/25/2019


In [47]:
## Double check re-assigned values
unidentified_agefrom_test_df = unidentified_df.loc[unidentified_df['Age From']==0]
unidentified_agefrom_test_df
# len(unidentified_agefrom_test_df)

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified
0,UP51903,3/30/1986,0.0,NaN,Marbury,Autauga,Alabama,Female,White / Caucasian,1/28/2020
3,UP55409,3/26/2000,0.0,NaN,NaN,Baldwin,Alabama,Male,"White / Caucasian, Asian, American Indian / Al...",2/13/2020
34,UP51245,11/29/2017,0.0,NaN,Huntsville,Madison,Alabama,Male,Uncertain,12/27/2019
35,UP11867,12/8/2013,0.0,NaN,Huntsville,Madison,Alabama,Male,White / Caucasian,11/9/2018
36,UP51243,11/29/2017,0.0,NaN,Huntsville,Madison,Alabama,Female,Uncertain,12/27/2019
...,...,...,...,...,...,...,...,...,...,...
13236,UP13617,11/14/2014,0.0,1.0,West Allis,Milwaukee,Wisconsin,Male,Uncertain,4/20/2016
13240,UP11611,6/5/2008,0.0,NaN,Baraboo,Sauk,Wisconsin,Male,Uncertain,11/22/2013
13248,UP7631,7/10/1973,0.0,NaN,Waukesha,Waukesha,Wisconsin,Male,White / Caucasian,11/9/2013
13251,UP58757,12/16/1970,0.0,NaN,Neenah,Winnebago,Wisconsin,Female,White / Caucasian,1/10/2020


In [48]:
## Check if any rows with null value for Age To
unidentified_attr_2_df = unidentified_df.loc[unidentified_df['Age To'].isnull()]
# len(unidentified_attr_2_df)
unidentified_attr_2_df

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified
0,UP51903,3/30/1986,0.0,NaN,Marbury,Autauga,Alabama,Female,White / Caucasian,1/28/2020
3,UP55409,3/26/2000,0.0,NaN,NaN,Baldwin,Alabama,Male,"White / Caucasian, Asian, American Indian / Al...",2/13/2020
34,UP51245,11/29/2017,0.0,NaN,Huntsville,Madison,Alabama,Male,Uncertain,12/27/2019
35,UP11867,12/8/2013,0.0,NaN,Huntsville,Madison,Alabama,Male,White / Caucasian,11/9/2018
36,UP51243,11/29/2017,0.0,NaN,Huntsville,Madison,Alabama,Female,Uncertain,12/27/2019
...,...,...,...,...,...,...,...,...,...,...
13206,UP52045,6/14/2018,0.0,NaN,Cyclone,Wyoming,West Virginia,Male,Uncertain,1/15/2020
13216,UP13695,11/9/1989,0.0,NaN,Paris,Kenosha,Wisconsin,Male,White / Caucasian,4/11/2017
13240,UP11611,6/5/2008,0.0,NaN,Baraboo,Sauk,Wisconsin,Male,Uncertain,11/22/2013
13248,UP7631,7/10/1973,0.0,NaN,Waukesha,Waukesha,Wisconsin,Male,White / Caucasian,11/9/2013


In [49]:
# Re-assign missing age to values to be 120
unidentified_df['Age To'] = unidentified_df['Age To'].fillna(120)
unidentified_df.head()

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified
0,UP51903,3/30/1986,0.0,120.0,Marbury,Autauga,Alabama,Female,White / Caucasian,1/28/2020
1,UP5000,9/29/2007,40.0,60.0,Prattville,Autauga,Alabama,Female,Black / African American,8/12/2019
2,UP6519,1/6/2006,45.0,59.0,Elberta,Baldwin,Alabama,Male,White / Caucasian,10/24/2018
3,UP55409,3/26/2000,0.0,120.0,NaN,Baldwin,Alabama,Male,"White / Caucasian, Asian, American Indian / Al...",2/13/2020
4,UP13483,3/27/1961,14.0,17.0,NaN,Bibb,Alabama,Male,White / Caucasian,3/25/2019


In [50]:
## Double check re-assigned values
unidentified_ageto_test_df = unidentified_df.loc[unidentified_df['Age To']==120]
unidentified_ageto_test_df
# len(unidentified_ageto_test_df)

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified
0,UP51903,3/30/1986,0.0,120.0,Marbury,Autauga,Alabama,Female,White / Caucasian,1/28/2020
3,UP55409,3/26/2000,0.0,120.0,NaN,Baldwin,Alabama,Male,"White / Caucasian, Asian, American Indian / Al...",2/13/2020
34,UP51245,11/29/2017,0.0,120.0,Huntsville,Madison,Alabama,Male,Uncertain,12/27/2019
35,UP11867,12/8/2013,0.0,120.0,Huntsville,Madison,Alabama,Male,White / Caucasian,11/9/2018
36,UP51243,11/29/2017,0.0,120.0,Huntsville,Madison,Alabama,Female,Uncertain,12/27/2019
...,...,...,...,...,...,...,...,...,...,...
13206,UP52045,6/14/2018,0.0,120.0,Cyclone,Wyoming,West Virginia,Male,Uncertain,1/15/2020
13216,UP13695,11/9/1989,0.0,120.0,Paris,Kenosha,Wisconsin,Male,White / Caucasian,4/11/2017
13240,UP11611,6/5/2008,0.0,120.0,Baraboo,Sauk,Wisconsin,Male,Uncertain,11/22/2013
13248,UP7631,7/10/1973,0.0,120.0,Waukesha,Waukesha,Wisconsin,Male,White / Caucasian,11/9/2013


In [51]:
## Check if any rows with null value for sex
unidentified_attr_3_df = unidentified_df.loc[unidentified_df['Sex'].isnull()]
# len(unidentified_attr_3_df)
unidentified_attr_3_df

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified
1621,UP54136,NaN,0.0,120.0,NaN,Pima,Arizona,NaN,Uncertain,1/16/2020


In [52]:
# Re-assign missing sex values to be 'Unknown'
unidentified_df['Sex'] = unidentified_df['Sex'].fillna('Unknown')
unidentified_df.head()

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified
0,UP51903,3/30/1986,0.0,120.0,Marbury,Autauga,Alabama,Female,White / Caucasian,1/28/2020
1,UP5000,9/29/2007,40.0,60.0,Prattville,Autauga,Alabama,Female,Black / African American,8/12/2019
2,UP6519,1/6/2006,45.0,59.0,Elberta,Baldwin,Alabama,Male,White / Caucasian,10/24/2018
3,UP55409,3/26/2000,0.0,120.0,NaN,Baldwin,Alabama,Male,"White / Caucasian, Asian, American Indian / Al...",2/13/2020
4,UP13483,3/27/1961,14.0,17.0,NaN,Bibb,Alabama,Male,White / Caucasian,3/25/2019


In [53]:
## Double check re-assigned values
unidentified_sex_test_df = unidentified_df.loc[unidentified_df['Sex']=='Unknown']
unidentified_sex_test_df
# len(unidentified_sex_test_df)

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified
1621,UP54136,NaN,0.0,120.0,NaN,Pima,Arizona,Unknown,Uncertain,1/16/2020


In [54]:
## Check if any rows with null value for race / ethnicity
unidentified_attr_4_df = unidentified_df.loc[unidentified_df['Race / Ethnicity'].isnull()]
# len(unidentified_attr_4_df)
unidentified_attr_4_df

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified
369,UP1978,11/10/2001,0.0,0.0,Gila Bend,Maricopa,Arizona,Female,NaN,7/26/2018
484,UP2024,12/1/2003,17.0,24.0,NaN,Maricopa,Arizona,Male,NaN,6/4/2014
499,UP1938,5/27/1996,22.0,43.0,NaN,Maricopa,Arizona,Male,NaN,6/4/2014
500,UP1936,2/27/1996,19.0,35.0,NaN,Maricopa,Arizona,Male,NaN,6/4/2014
506,UP1927,12/7/1991,45.0,60.0,NaN,Maricopa,Arizona,Male,NaN,4/4/2019
511,UP1914,9/23/1987,25.0,35.0,NaN,Maricopa,Arizona,Male,NaN,6/4/2014
4527,UP5903,5/9/1984,0.0,0.0,NaN,San Francisco,California,Male,NaN,7/23/2019
4528,UP5874,11/27/1981,0.0,0.0,NaN,San Francisco,California,Male,NaN,7/23/2019
4529,UP5841,3/7/1975,0.0,0.0,NaN,San Francisco,California,Male,NaN,7/23/2019
4656,UP53055,4/27/1986,0.0,120.0,Porterville,Tulare,California,Male,NaN,11/29/2018


In [55]:
# Re-assign missing race / ethnicity values to be 'Uncertain'
unidentified_df['Race / Ethnicity'] = unidentified_df['Race / Ethnicity'].fillna('Uncertain')
unidentified_df.head()

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified
0,UP51903,3/30/1986,0.0,120.0,Marbury,Autauga,Alabama,Female,White / Caucasian,1/28/2020
1,UP5000,9/29/2007,40.0,60.0,Prattville,Autauga,Alabama,Female,Black / African American,8/12/2019
2,UP6519,1/6/2006,45.0,59.0,Elberta,Baldwin,Alabama,Male,White / Caucasian,10/24/2018
3,UP55409,3/26/2000,0.0,120.0,NaN,Baldwin,Alabama,Male,"White / Caucasian, Asian, American Indian / Al...",2/13/2020
4,UP13483,3/27/1961,14.0,17.0,NaN,Bibb,Alabama,Male,White / Caucasian,3/25/2019


In [56]:
## Double check re-assigned values
unidentified_race_test_df = unidentified_df.loc[unidentified_df['Race / Ethnicity']=='Uncertain']
unidentified_race_test_df
# len(unidentified_race_test_df)

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified
6,UP5011,7/22/2007,30.0,40.0,Titus,Elmore,Alabama,Female,Uncertain,4/13/2020
7,UP15338,4/12/2002,18.0,99.0,Adger,Jefferson,Alabama,Unsure,Uncertain,3/18/2020
11,UP14991,4/8/2016,20.0,60.0,Birmingham,Jefferson,Alabama,Male,Uncertain,3/18/2020
12,UP13987,12/23/2012,18.0,99.0,Birmingham,Jefferson,Alabama,Male,Uncertain,3/18/2020
15,UP14466,6/22/2004,17.0,30.0,Birmingham,Jefferson,Alabama,Male,Uncertain,3/18/2020
...,...,...,...,...,...,...,...,...,...,...
13243,UP12012,10/11/1979,20.0,35.0,East Troy,Walworth,Wisconsin,Male,Uncertain,7/18/2019
13245,UP7405,11/18/1984,30.0,55.0,Town of Mukwonago,Waukesha,Wisconsin,Female,Uncertain,10/11/2019
13246,UP7634,9/28/1975,18.0,99.0,Town of Oconomowoc,Waukesha,Wisconsin,Male,Uncertain,1/27/2012
13249,UP7548,4/3/1976,25.0,50.0,Waukesha,Waukesha,Wisconsin,Female,Uncertain,10/3/2019


# Part 2 - Prep data for state-level GeoJSON with data from all 3 databases

### 2a: Add in State FIPS column to each database dataframe

In [57]:
# make dictionary of states and state FIPS code
state_dict = dict(zip(state_centroids_df.NAME, state_centroids_df.STATEFP))
# state_dict
# state_dict['Alaska']
len(state_dict)

55

#### 1/3: add state FIPS codes to missing persons dataframe

In [58]:
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified
0,MP2341,5/8/1999,Reynolds,James,28,Opp,Autauga,Alabama,Male,Black / African American,2/14/2020
1,MP8522,1/15/2010,Walker,Jeremy,32,Prattville,Autauga,Alabama,Male,White / Caucasian,7/27/2015
2,MP50314,5/16/2018,Keszthelyi,Steve,79,Elberta,Baldwin,Alabama,Male,White / Caucasian,11/13/2019
3,MP43187,2/20/2018,Johnson,Abram,60,fairhope,Baldwin,Alabama,Male,White / Caucasian,2/13/2020
4,MP23019,11/5/2012,Shroyer,James,40,Fairhope,Baldwin,Alabama,Male,White / Caucasian,2/3/2016


In [59]:
# Add new column to dataframe, using the state name field as a key in the state_dict, to pull the correct FIPS code for each row
missing_df['State_FIPS'] = missing_df['State'].map(state_dict)
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS
0,MP2341,5/8/1999,Reynolds,James,28,Opp,Autauga,Alabama,Male,Black / African American,2/14/2020,1
1,MP8522,1/15/2010,Walker,Jeremy,32,Prattville,Autauga,Alabama,Male,White / Caucasian,7/27/2015,1
2,MP50314,5/16/2018,Keszthelyi,Steve,79,Elberta,Baldwin,Alabama,Male,White / Caucasian,11/13/2019,1
3,MP43187,2/20/2018,Johnson,Abram,60,fairhope,Baldwin,Alabama,Male,White / Caucasian,2/13/2020,1
4,MP23019,11/5/2012,Shroyer,James,40,Fairhope,Baldwin,Alabama,Male,White / Caucasian,2/3/2016,1


In [60]:
# Check unique values in new dataframe field
missing_df['State_FIPS'].unique()

array([ 1,  2,  4,  5,  6,  8, 69,  9, 10, 11, 12, 13, 66, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 72, 44, 45, 46, 47, 48, 78, 49, 50, 51,
       53, 54, 55, 56], dtype=int64)

In [61]:
## If any nulls, check to see what is producing null values [NOTE: fixed by making changes to csv fields, as noted where csvs are imported]
mis_test_df = missing_df.loc[missing_df['State_FIPS'].isnull()]
mis_test_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS


In [62]:
# check missing df
missing_df.head()
# len(missing_df)

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS
0,MP2341,5/8/1999,Reynolds,James,28,Opp,Autauga,Alabama,Male,Black / African American,2/14/2020,1
1,MP8522,1/15/2010,Walker,Jeremy,32,Prattville,Autauga,Alabama,Male,White / Caucasian,7/27/2015,1
2,MP50314,5/16/2018,Keszthelyi,Steve,79,Elberta,Baldwin,Alabama,Male,White / Caucasian,11/13/2019,1
3,MP43187,2/20/2018,Johnson,Abram,60,fairhope,Baldwin,Alabama,Male,White / Caucasian,2/13/2020,1
4,MP23019,11/5/2012,Shroyer,James,40,Fairhope,Baldwin,Alabama,Male,White / Caucasian,2/3/2016,1


#### 2/3: add state FIPS codes to unclaimed persons dataframe

In [63]:
unclaimed_df.head()
# len(unclaimed_df)

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified
0,UCP68231,4/15/2020,Sanders,Stephen,Male,White / Caucasian,Joliet,Will,Illinois,4/16/2020
1,UCP68248,4/13/2020,Brookshire,Renee,Female,White / Caucasian,Joliet,Will,Illinois,4/16/2020
2,UCP68242,4/9/2020,Smith,Victoria,Female,White / Caucasian,Manhattan,New York,New York,4/16/2020
3,UCP68244,4/9/2020,Bellamy,Eldred,Male,Black / African American,Brooklyn,Kings,New York,4/16/2020
4,UCP68228,4/9/2020,Rodriguez,Elido,Male,Hispanic / Latino,Manhattan,New York,New York,4/15/2020


In [64]:
# Add new column to dataframe, using the state name field as a key in the state_dict, to pull the correct FIPS code for each row
unclaimed_df['State_FIPS'] = unclaimed_df['State'].map(state_dict)
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS
0,UCP68231,4/15/2020,Sanders,Stephen,Male,White / Caucasian,Joliet,Will,Illinois,4/16/2020,17.0
1,UCP68248,4/13/2020,Brookshire,Renee,Female,White / Caucasian,Joliet,Will,Illinois,4/16/2020,17.0
2,UCP68242,4/9/2020,Smith,Victoria,Female,White / Caucasian,Manhattan,New York,New York,4/16/2020,36.0
3,UCP68244,4/9/2020,Bellamy,Eldred,Male,Black / African American,Brooklyn,Kings,New York,4/16/2020,36.0
4,UCP68228,4/9/2020,Rodriguez,Elido,Male,Hispanic / Latino,Manhattan,New York,New York,4/15/2020,36.0


In [65]:
# Check unique values in new dataframe field
unclaimed_df['State_FIPS'].unique()

array([17., 36., 25.,  6., 53., 15., 32., 16., 48., 49., 47., 11., 33.,
       35., 40., 42., 39., 29.,  5., 13.,  1., 34.,  4., 12., 55.,  9.,
        8., 26., 22., 18., 19., 41., 23., nan, 45., 56., 54., 51.,  2.,
       28., 21., 37., 10., 38.])

In [66]:
## If any nulls, check to see what is producing null values [NOTE: cannot be addressed, as these cases do not have a state or county assigned]
unc_test_df = unclaimed_df.loc[unclaimed_df['State_FIPS'].isnull()]
# len(unc_test_df)
unc_test_df

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS
3380,UCP5339,12/3/2016,Williams,Barbara,Female,Asian,NaN,NaN,NaN,5/3/2018,NaN
4774,UCP2489,3/26/2015,Lupien,Daryl,Male,Uncertain,NaN,NaN,NaN,8/18/2016,NaN
5023,UCP4001,11/2/2014,Brown,David,Male,White / Caucasian,NaN,NaN,NaN,5/11/2017,NaN
5883,UCP3962,3/22/2010,Gonzalez,David,Male,Hispanic / Latino,NaN,NaN,NaN,5/11/2017,NaN
5950,UCP3937,12/7/2009,Togovnick,Bernice,Female,White / Caucasian,NaN,NaN,NaN,5/11/2017,NaN
6183,UCP3901,1/21/2009,Carney,John,Male,White / Caucasian,NaN,NaN,NaN,5/11/2017,NaN
6341,UCP3689,6/5/2008,Guzman,Ralph,Male,Hispanic / Latino,NaN,NaN,NaN,5/2/2017,NaN
6560,UCP3824,7/2/2007,Vasquez,Rafael,Male,Hispanic / Latino,NaN,NaN,NaN,5/4/2017,NaN
6698,UCP3672,9/22/2006,Barajas,Epifanio,Male,Hispanic / Latino,NaN,NaN,NaN,5/2/2017,NaN
6719,UCP3668,8/5/2006,Huff,Paul,Male,White / Caucasian,NaN,NaN,NaN,5/2/2017,NaN


In [67]:
# # change NaN FIPS codes (for cases with no city, county or state) to 99 (none option in state_centroids_v2.csv)
unclaimed_df['State_FIPS'] = unclaimed_df['State_FIPS'].fillna(99)
unclaimed_df['State_FIPS'] = unclaimed_df['State_FIPS'].astype(int)
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS
0,UCP68231,4/15/2020,Sanders,Stephen,Male,White / Caucasian,Joliet,Will,Illinois,4/16/2020,17
1,UCP68248,4/13/2020,Brookshire,Renee,Female,White / Caucasian,Joliet,Will,Illinois,4/16/2020,17
2,UCP68242,4/9/2020,Smith,Victoria,Female,White / Caucasian,Manhattan,New York,New York,4/16/2020,36
3,UCP68244,4/9/2020,Bellamy,Eldred,Male,Black / African American,Brooklyn,Kings,New York,4/16/2020,36
4,UCP68228,4/9/2020,Rodriguez,Elido,Male,Hispanic / Latino,Manhattan,New York,New York,4/15/2020,36


In [68]:
## Double check re-assigned values
unc_test_df2 = unclaimed_df.loc[unclaimed_df['State_FIPS']==99]
unc_test_df2
# len(unc_test_df2)

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS
3380,UCP5339,12/3/2016,Williams,Barbara,Female,Asian,NaN,NaN,NaN,5/3/2018,99
4774,UCP2489,3/26/2015,Lupien,Daryl,Male,Uncertain,NaN,NaN,NaN,8/18/2016,99
5023,UCP4001,11/2/2014,Brown,David,Male,White / Caucasian,NaN,NaN,NaN,5/11/2017,99
5883,UCP3962,3/22/2010,Gonzalez,David,Male,Hispanic / Latino,NaN,NaN,NaN,5/11/2017,99
5950,UCP3937,12/7/2009,Togovnick,Bernice,Female,White / Caucasian,NaN,NaN,NaN,5/11/2017,99
6183,UCP3901,1/21/2009,Carney,John,Male,White / Caucasian,NaN,NaN,NaN,5/11/2017,99
6341,UCP3689,6/5/2008,Guzman,Ralph,Male,Hispanic / Latino,NaN,NaN,NaN,5/2/2017,99
6560,UCP3824,7/2/2007,Vasquez,Rafael,Male,Hispanic / Latino,NaN,NaN,NaN,5/4/2017,99
6698,UCP3672,9/22/2006,Barajas,Epifanio,Male,Hispanic / Latino,NaN,NaN,NaN,5/2/2017,99
6719,UCP3668,8/5/2006,Huff,Paul,Male,White / Caucasian,NaN,NaN,NaN,5/2/2017,99


In [69]:
# check full dataframe
len(unclaimed_df)

8335

#### 3/3: add state FIPS codes to unidentified persons dataframe

In [70]:
unidentified_df.head()

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified
0,UP51903,3/30/1986,0.0,120.0,Marbury,Autauga,Alabama,Female,White / Caucasian,1/28/2020
1,UP5000,9/29/2007,40.0,60.0,Prattville,Autauga,Alabama,Female,Black / African American,8/12/2019
2,UP6519,1/6/2006,45.0,59.0,Elberta,Baldwin,Alabama,Male,White / Caucasian,10/24/2018
3,UP55409,3/26/2000,0.0,120.0,NaN,Baldwin,Alabama,Male,"White / Caucasian, Asian, American Indian / Al...",2/13/2020
4,UP13483,3/27/1961,14.0,17.0,NaN,Bibb,Alabama,Male,White / Caucasian,3/25/2019


In [71]:
# Add new column to dataframe, using the state name field as a key in the state_dict, to pull the correct FIPS code for each row
unidentified_df['State_FIPS'] = unidentified_df['State'].map(state_dict)
unidentified_df.head()

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS
0,UP51903,3/30/1986,0.0,120.0,Marbury,Autauga,Alabama,Female,White / Caucasian,1/28/2020,1
1,UP5000,9/29/2007,40.0,60.0,Prattville,Autauga,Alabama,Female,Black / African American,8/12/2019,1
2,UP6519,1/6/2006,45.0,59.0,Elberta,Baldwin,Alabama,Male,White / Caucasian,10/24/2018,1
3,UP55409,3/26/2000,0.0,120.0,NaN,Baldwin,Alabama,Male,"White / Caucasian, Asian, American Indian / Al...",2/13/2020,1
4,UP13483,3/27/1961,14.0,17.0,NaN,Bibb,Alabama,Male,White / Caucasian,3/25/2019,1


In [72]:
# Check unique values in new dataframe field
unidentified_df['State_FIPS'].unique()

array([ 1,  2,  4,  5,  6,  8,  9, 10, 11, 12, 13, 66, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 72, 44, 45, 46, 47, 48, 78, 49, 50, 51, 53,
       54, 55, 56], dtype=int64)

In [73]:
## If any nulls, check to see what is producing null values [NOTE: fixed by making changes to csv fields, as noted where csvs are imported]
uni_test_df = unidentified_df.loc[unidentified_df['State_FIPS'].isnull()]
uni_test_df.head()

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS


# Part 3: Prep data for county-level JSONs for all 3 databases

#### Add in State name and County_Key columns to county centroid v2 dataframe

In [74]:
# make dictionary of state FIPS codes and state names
state_FIPS_dict = dict(zip(state_centroids_v2_df.STATEFP, state_centroids_v2_df.NAME))
# state_FIPS_dict
# state_FIPS_dict['Alaska']
len(state_FIPS_dict)

56

In [75]:
# check county centroids v2 df
county_centroids_v2_df.head()

,OBJECTID,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,ORIG_FID,Lon_dd,Lat_dd
0,1,21,7,516850,0500000US21007,21007,Ballard,6,6.393875e+08,69473325,1,-88.999262,37.058489
1,2,21,17,516855,0500000US21017,21017,Bourbon,6,7.504394e+08,4829777,2,-84.217155,38.206742
2,3,21,31,516862,0500000US21031,21031,Butler,6,1.103572e+09,13943044,3,-86.681628,37.207292
3,4,21,65,516879,0500000US21065,21065,Estill,6,6.555099e+08,6516335,4,-83.964316,37.692451
4,5,21,69,516881,0500000US21069,21069,Fleming,6,9.027272e+08,7182793,5,-83.696660,38.370126


In [76]:
# Add new column to dataframe, using the state name field as a key in the state_dict, to pull the correct FIPS code for each row
county_centroids_v2_df['STATE_NAME'] = county_centroids_v2_df['STATEFP'].map(state_FIPS_dict)
# check dataframe
county_centroids_v2_df.head()
# Check unique values in new dataframe field
# county_centroids_v2_df['STATE_NAME'].unique()

,OBJECTID,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,ORIG_FID,Lon_dd,Lat_dd,STATE_NAME
0,1,21,7,516850,0500000US21007,21007,Ballard,6,6.393875e+08,69473325,1,-88.999262,37.058489,Kentucky
1,2,21,17,516855,0500000US21017,21017,Bourbon,6,7.504394e+08,4829777,2,-84.217155,38.206742,Kentucky
2,3,21,31,516862,0500000US21031,21031,Butler,6,1.103572e+09,13943044,3,-86.681628,37.207292,Kentucky
3,4,21,65,516879,0500000US21065,21065,Estill,6,6.555099e+08,6516335,4,-83.964316,37.692451,Kentucky
4,5,21,69,516881,0500000US21069,21069,Fleming,6,9.027272e+08,7182793,5,-83.696660,38.370126,Kentucky


In [77]:
# sort county centroids by state FIPS
county_centroids_v2_df = county_centroids_v2_df.sort_values(by=['STATEFP'])
county_centroids_v2_df.head()

,OBJECTID,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,ORIG_FID,Lon_dd,Lat_dd,STATE_NAME
1043,1044,1,61,161556,0500000US01061,1061,Geneva,6,1.487898e+09,11578163,1044,-85.839096,31.095061,Alabama
1255,1256,1,17,161534,0500000US01017,1017,Chambers,6,1.545086e+09,16971701,1256,-85.392035,32.914352,Alabama
3228,100000,1,999,9999100,9999999US100000,1999,None,0,0.000000e+00,0,99991,45.000000,-90.000000,Alabama
471,472,1,49,161550,0500000US01049,1049,DeKalb,6,2.012676e+09,4121543,472,-85.804114,34.459807,Alabama
472,473,1,63,161557,0500000US01063,1063,Greene,6,1.675782e+09,33416141,473,-87.952209,32.853154,Alabama


In [78]:
# Add column with compound field key
county_centroids_v2_df['County_Key'] = county_centroids_v2_df['STATEFP'].astype(str) + "_" + county_centroids_v2_df['NAME']
county_centroids_v2_df.head()

,OBJECTID,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,ORIG_FID,Lon_dd,Lat_dd,STATE_NAME,County_Key
1043,1044,1,61,161556,0500000US01061,1061,Geneva,6,1.487898e+09,11578163,1044,-85.839096,31.095061,Alabama,1_Geneva
1255,1256,1,17,161534,0500000US01017,1017,Chambers,6,1.545086e+09,16971701,1256,-85.392035,32.914352,Alabama,1_Chambers
3228,100000,1,999,9999100,9999999US100000,1999,None,0,0.000000e+00,0,99991,45.000000,-90.000000,Alabama,1_None
471,472,1,49,161550,0500000US01049,1049,DeKalb,6,2.012676e+09,4121543,472,-85.804114,34.459807,Alabama,1_DeKalb
472,473,1,63,161557,0500000US01063,1063,Greene,6,1.675782e+09,33416141,473,-87.952209,32.853154,Alabama,1_Greene


In [79]:
# check unique values and length
county_key_v2_list = county_centroids_v2_df['County_Key'].unique()
len(county_key_v2_list)

3283

In [80]:
# make dictionary of counties and county FIPS code (GEOID field)
county_v2_dict = dict(zip(county_centroids_v2_df.County_Key, county_centroids_v2_df.GEOID))
len(county_v2_dict)

3283

#### Add in County_key column to county centroids dataframe

In [81]:
# sort county centroids by state FIPS
county_centroids_df = county_centroids_df.sort_values(by=['STATEFP'])
county_centroids_df.head()

,OBJECTID,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,ORIG_FID,Lon_dd,Lat_dd
3211,3212,1,121,161586,0500000US01121,1121,Talladega,6,1.908257e+09,60926366,3218,-86.165890,33.380086
1194,1195,1,93,161573,0500000US01093,1093,Marion,6,1.922657e+09,3184076,1195,-87.887140,34.136562
1255,1256,1,17,161534,0500000US01017,1017,Chambers,6,1.545086e+09,16971701,1256,-85.392035,32.914352
1215,1216,1,59,161555,0500000US01059,1059,Franklin,6,1.641841e+09,32643981,1216,-87.843740,34.441699
2780,2781,1,127,161589,0500000US01127,1127,Walker,6,2.048686e+09,36754696,2785,-87.297329,33.803318


In [82]:
# Add column with compound field key
county_centroids_df['County_Key'] = county_centroids_df['STATEFP'].astype(str) + "_" + county_centroids_df['NAME']
county_centroids_df.head()

,OBJECTID,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,ORIG_FID,Lon_dd,Lat_dd,County_Key
3211,3212,1,121,161586,0500000US01121,1121,Talladega,6,1.908257e+09,60926366,3218,-86.165890,33.380086,1_Talladega
1194,1195,1,93,161573,0500000US01093,1093,Marion,6,1.922657e+09,3184076,1195,-87.887140,34.136562,1_Marion
1255,1256,1,17,161534,0500000US01017,1017,Chambers,6,1.545086e+09,16971701,1256,-85.392035,32.914352,1_Chambers
1215,1216,1,59,161555,0500000US01059,1059,Franklin,6,1.641841e+09,32643981,1216,-87.843740,34.441699,1_Franklin
2780,2781,1,127,161589,0500000US01127,1127,Walker,6,2.048686e+09,36754696,2785,-87.297329,33.803318,1_Walker


In [83]:
# check unique values and length
county_key_list = county_centroids_df['County_Key'].unique()
len(county_key_list)

3227

In [84]:
# make dictionary of counties and county FIPS code (GEOID field)
county_dict = dict(zip(county_centroids_df.County_Key, county_centroids_df.GEOID))
len(county_dict)

3227

#### Add in County FIPS column to missing dataframe

##### To identify null values, add in County FIPS column to missing dataframe using county_centroids_df

In [85]:
# check missing df
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS
0,MP2341,5/8/1999,Reynolds,James,28,Opp,Autauga,Alabama,Male,Black / African American,2/14/2020,1
1,MP8522,1/15/2010,Walker,Jeremy,32,Prattville,Autauga,Alabama,Male,White / Caucasian,7/27/2015,1
2,MP50314,5/16/2018,Keszthelyi,Steve,79,Elberta,Baldwin,Alabama,Male,White / Caucasian,11/13/2019,1
3,MP43187,2/20/2018,Johnson,Abram,60,fairhope,Baldwin,Alabama,Male,White / Caucasian,2/13/2020,1
4,MP23019,11/5/2012,Shroyer,James,40,Fairhope,Baldwin,Alabama,Male,White / Caucasian,2/3/2016,1


In [86]:
# Add column with compound field key
missing_df['State_County'] = missing_df['State_FIPS'].astype(str) + "_" + missing_df['County']
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County
0,MP2341,5/8/1999,Reynolds,James,28,Opp,Autauga,Alabama,Male,Black / African American,2/14/2020,1,1_Autauga
1,MP8522,1/15/2010,Walker,Jeremy,32,Prattville,Autauga,Alabama,Male,White / Caucasian,7/27/2015,1,1_Autauga
2,MP50314,5/16/2018,Keszthelyi,Steve,79,Elberta,Baldwin,Alabama,Male,White / Caucasian,11/13/2019,1,1_Baldwin
3,MP43187,2/20/2018,Johnson,Abram,60,fairhope,Baldwin,Alabama,Male,White / Caucasian,2/13/2020,1,1_Baldwin
4,MP23019,11/5/2012,Shroyer,James,40,Fairhope,Baldwin,Alabama,Male,White / Caucasian,2/3/2016,1,1_Baldwin


In [87]:
# Add new column to dataframe, using the State_County name field as a key in the county_dict, to pull the correct County FIPS code for each row
missing_df['County_FIPS'] = missing_df['State_County'].map(county_dict)
# Check unique values in new dataframe field
missing_df['County_FIPS'].unique()

array([ 1001.,  1003.,  1005., ..., 56035., 56037., 56039.])

In [88]:
# check null values [NOTE: all ok - no city or county assigned]
missing_county_null_df = missing_df.loc[missing_df['County_FIPS'].isnull()]
# missing_county_null_df.shape
missing_county_null_df

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS
14181,MP4569,5/5/1974,Colonna aponte,John,12,NaN,NaN,Puerto Rico,Male,"White / Caucasian, Hispanic / Latino",5/15/2018,72,NaN,NaN
14182,MP54374,12/15/1984,Cruz,Marcos,2,NaN,NaN,Puerto Rico,Male,Black / African American,3/12/2020,72,NaN,NaN
14183,MP4568,5/4/1974,Colonna aponte,Giannina,11,NaN,NaN,Puerto Rico,Female,"White / Caucasian, Hispanic / Latino",4/25/2018,72,NaN,NaN


In [89]:
# As needed, export nulls to address [NOTE: all that can have been addressed]
# missing_county_null_df.to_csv('missing_county_nulls.csv', encoding='Windows-1252')

In [90]:
# check missing_df
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS
0,MP2341,5/8/1999,Reynolds,James,28,Opp,Autauga,Alabama,Male,Black / African American,2/14/2020,1,1_Autauga,1001.0
1,MP8522,1/15/2010,Walker,Jeremy,32,Prattville,Autauga,Alabama,Male,White / Caucasian,7/27/2015,1,1_Autauga,1001.0
2,MP50314,5/16/2018,Keszthelyi,Steve,79,Elberta,Baldwin,Alabama,Male,White / Caucasian,11/13/2019,1,1_Baldwin,1003.0
3,MP43187,2/20/2018,Johnson,Abram,60,fairhope,Baldwin,Alabama,Male,White / Caucasian,2/13/2020,1,1_Baldwin,1003.0
4,MP23019,11/5/2012,Shroyer,James,40,Fairhope,Baldwin,Alabama,Male,White / Caucasian,2/3/2016,1,1_Baldwin,1003.0


##### Replace null state values with 'None' and null county values with 'None', then re-write county_FIPS column using county_centroids_v2_dataframe

In [91]:
## Check if any rows with null value for 'State'
mis_state_test_df = missing_df.loc[missing_df['State'].isnull()]
# len(mis_state_test_df)
mis_state_test_df

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS


In [92]:
## Check if any rows with null value for 'City' and County'
mis_county_test_df = missing_df.loc[missing_df['County'].isnull()]
# len(mis_county_test_df)
mis_county_test_df

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS
14181,MP4569,5/5/1974,Colonna aponte,John,12,NaN,NaN,Puerto Rico,Male,"White / Caucasian, Hispanic / Latino",5/15/2018,72,NaN,NaN
14182,MP54374,12/15/1984,Cruz,Marcos,2,NaN,NaN,Puerto Rico,Male,Black / African American,3/12/2020,72,NaN,NaN
14183,MP4568,5/4/1974,Colonna aponte,Giannina,11,NaN,NaN,Puerto Rico,Female,"White / Caucasian, Hispanic / Latino",4/25/2018,72,NaN,NaN


In [93]:
# # change NaN County name (for cases with no city or county) to 'None'
missing_df['County'] = missing_df['County'].fillna('None')
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS
0,MP2341,5/8/1999,Reynolds,James,28,Opp,Autauga,Alabama,Male,Black / African American,2/14/2020,1,1_Autauga,1001.0
1,MP8522,1/15/2010,Walker,Jeremy,32,Prattville,Autauga,Alabama,Male,White / Caucasian,7/27/2015,1,1_Autauga,1001.0
2,MP50314,5/16/2018,Keszthelyi,Steve,79,Elberta,Baldwin,Alabama,Male,White / Caucasian,11/13/2019,1,1_Baldwin,1003.0
3,MP43187,2/20/2018,Johnson,Abram,60,fairhope,Baldwin,Alabama,Male,White / Caucasian,2/13/2020,1,1_Baldwin,1003.0
4,MP23019,11/5/2012,Shroyer,James,40,Fairhope,Baldwin,Alabama,Male,White / Caucasian,2/3/2016,1,1_Baldwin,1003.0


In [94]:
## Double check re-assigned values
mis_county_test_2_df = missing_df.loc[missing_df['County']=='None']
mis_county_test_2_df
# len(mis_county_test_2_df)

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS
14181,MP4569,5/5/1974,Colonna aponte,John,12,NaN,None,Puerto Rico,Male,"White / Caucasian, Hispanic / Latino",5/15/2018,72,NaN,NaN
14182,MP54374,12/15/1984,Cruz,Marcos,2,NaN,None,Puerto Rico,Male,Black / African American,3/12/2020,72,NaN,NaN
14183,MP4568,5/4/1974,Colonna aponte,Giannina,11,NaN,None,Puerto Rico,Female,"White / Caucasian, Hispanic / Latino",4/25/2018,72,NaN,NaN


In [95]:
# Re-Add column with compound field key
missing_df['State_County'] = missing_df['State_FIPS'].astype(str) + "_" + missing_df['County']
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS
0,MP2341,5/8/1999,Reynolds,James,28,Opp,Autauga,Alabama,Male,Black / African American,2/14/2020,1,1_Autauga,1001.0
1,MP8522,1/15/2010,Walker,Jeremy,32,Prattville,Autauga,Alabama,Male,White / Caucasian,7/27/2015,1,1_Autauga,1001.0
2,MP50314,5/16/2018,Keszthelyi,Steve,79,Elberta,Baldwin,Alabama,Male,White / Caucasian,11/13/2019,1,1_Baldwin,1003.0
3,MP43187,2/20/2018,Johnson,Abram,60,fairhope,Baldwin,Alabama,Male,White / Caucasian,2/13/2020,1,1_Baldwin,1003.0
4,MP23019,11/5/2012,Shroyer,James,40,Fairhope,Baldwin,Alabama,Male,White / Caucasian,2/3/2016,1,1_Baldwin,1003.0


In [96]:
## Double check re-assigned values
mis_county_test_3_df = missing_df.loc[missing_df['County']=='None']
mis_county_test_3_df
# len(mis_county_test_3_df)

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS
14181,MP4569,5/5/1974,Colonna aponte,John,12,NaN,None,Puerto Rico,Male,"White / Caucasian, Hispanic / Latino",5/15/2018,72,72_None,NaN
14182,MP54374,12/15/1984,Cruz,Marcos,2,NaN,None,Puerto Rico,Male,Black / African American,3/12/2020,72,72_None,NaN
14183,MP4568,5/4/1974,Colonna aponte,Giannina,11,NaN,None,Puerto Rico,Female,"White / Caucasian, Hispanic / Latino",4/25/2018,72,72_None,NaN


In [97]:
# Re-add County FIPS column to dataframe, using the State_County name field as a key in the county_v2_dict, to pull the correct County FIPS code for each row
missing_df['County_FIPS'] = missing_df['State_County'].map(county_v2_dict)
# Check unique values in new dataframe field
missing_df['County_FIPS'].unique()

array([ 1001,  1003,  1005, ..., 56035, 56037, 56039], dtype=int64)

In [98]:
# check null values to make sure none are left [Note - all gone!]
missing_county_null_v2_df = missing_df.loc[missing_df['County_FIPS'].isnull()]
# missing_county_null_v2_df.shape
missing_county_null_v2_df

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS


In [99]:
## Double check re-assigned values
mis_county_test_4_df = missing_df.loc[missing_df['County']=='None']
mis_county_test_4_df
# len(mis_county_test_4_df)

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS
14181,MP4569,5/5/1974,Colonna aponte,John,12,NaN,None,Puerto Rico,Male,"White / Caucasian, Hispanic / Latino",5/15/2018,72,72_None,72999
14182,MP54374,12/15/1984,Cruz,Marcos,2,NaN,None,Puerto Rico,Male,Black / African American,3/12/2020,72,72_None,72999
14183,MP4568,5/4/1974,Colonna aponte,Giannina,11,NaN,None,Puerto Rico,Female,"White / Caucasian, Hispanic / Latino",4/25/2018,72,72_None,72999


#### Add in County FIPS column to unclaimed dataframe

##### To identify null values, add in County FIPS column to unclaimed dataframe using county_centroids_df

In [100]:
# check unclaimed df
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS
0,UCP68231,4/15/2020,Sanders,Stephen,Male,White / Caucasian,Joliet,Will,Illinois,4/16/2020,17
1,UCP68248,4/13/2020,Brookshire,Renee,Female,White / Caucasian,Joliet,Will,Illinois,4/16/2020,17
2,UCP68242,4/9/2020,Smith,Victoria,Female,White / Caucasian,Manhattan,New York,New York,4/16/2020,36
3,UCP68244,4/9/2020,Bellamy,Eldred,Male,Black / African American,Brooklyn,Kings,New York,4/16/2020,36
4,UCP68228,4/9/2020,Rodriguez,Elido,Male,Hispanic / Latino,Manhattan,New York,New York,4/15/2020,36


In [101]:
# Add column with compound field key
unclaimed_df['State_County'] = unclaimed_df['State_FIPS'].astype(str) + "_" + unclaimed_df['County']
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS,State_County
0,UCP68231,4/15/2020,Sanders,Stephen,Male,White / Caucasian,Joliet,Will,Illinois,4/16/2020,17,17_Will
1,UCP68248,4/13/2020,Brookshire,Renee,Female,White / Caucasian,Joliet,Will,Illinois,4/16/2020,17,17_Will
2,UCP68242,4/9/2020,Smith,Victoria,Female,White / Caucasian,Manhattan,New York,New York,4/16/2020,36,36_New York
3,UCP68244,4/9/2020,Bellamy,Eldred,Male,Black / African American,Brooklyn,Kings,New York,4/16/2020,36,36_Kings
4,UCP68228,4/9/2020,Rodriguez,Elido,Male,Hispanic / Latino,Manhattan,New York,New York,4/15/2020,36,36_New York


In [102]:
# Add new column to dataframe, using the state name field as a key in the state_dict, to pull the correct FIPS code for each row
unclaimed_df['County_FIPS'] = unclaimed_df['State_County'].map(county_dict)
# Check unique values in new dataframe field
unclaimed_df['County_FIPS'].unique()

array([17197., 36061., 36047., 36005., 36081., 25025.,  6073., 17063.,
       36085., 53053., 15003., 32031., 25021., 25027., 16001., 25009.,
       25017., 25003., 25013., 25023., 32027., 25015.,  6047., 48201.,
       49035., 47093., 53075., 53027., 11001., 25005., 47157., 33005.,
       35031., 53049., 53077., 40143., 42029., 47155., 47001., 39099.,
       29019.,  6075.,  5143.,  6093., 13121.,  1073., 34025., 34023.,
       25001.,  4019.,  4013.,  6023.,  6099., 34001., 12087., 47035.,
        6037., 40121., 53061., 55025.,  6071.,  9003., 48141., 48339.,
        9009., 42101., 12071., 53033., 17043., 34021.,  9011., 48215.,
        8087., 12051., 26163.,  9001., 47009., 17031., 22069.,  9005.,
       32005.,  9007., 47065., 34013.,    nan,  4007., 47013., 18067.,
       42001.,  6059., 19155.,  6083., 26025., 40101., 40041., 42091.,
       55057., 48167., 19153.,  6025., 26121., 40145., 25011.,  9015.,
        1115.,  5035., 53035., 53073., 41051., 16027., 23005., 47179.,
      

In [103]:
# check null values
unclaimed_county_null_df = unclaimed_df.loc[unclaimed_df['County_FIPS'].isnull()]
# unclaimed_county_null_df.shape
unclaimed_county_null_df

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS,State_County,County_FIPS
1996,UCP5438,4/30/2018,Flores,Victor,Male,Black / African American,NaN,NaN,New York,5/14/2019,36,NaN,NaN
2011,UCP5440,4/22/2018,Browne,Edward,Male,Hispanic / Latino,NaN,NaN,New York,5/27/2019,36,NaN,NaN
2028,UCP5297,4/15/2018,Davis,Thelma,Female,Black / African American,NaN,NaN,New York,5/20/2019,36,NaN,NaN
2031,UCP5296,4/14/2018,Fasulo,Timothy,Male,White / Caucasian,NaN,NaN,New York,4/25/2019,36,NaN,NaN
2033,UCP5439,4/14/2018,Reyes,Nivia,Female,Other,NaN,NaN,New York,5/27/2019,36,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7286,UCP990,11/6/1997,Pope,Leon,Male,White / Caucasian,NaN,NaN,Washington,4/14/2020,53,NaN,NaN
7339,UCP1999,10/22/1991,Robles,Salvador,Male,Other,NaN,NaN,Tennessee,1/10/2020,47,NaN,NaN
7344,UCP4522,10/18/1990,Way,Kenneth,Male,Black / African American,NaN,NaN,New York,5/11/2019,36,NaN,NaN
7372,UCP1548,6/17/1973,Seah,Tee kee,Male,Asian,NaN,NaN,Nevada,5/28/2019,32,NaN,NaN


In [104]:
# As needed, export nulls to address [NOTE: all have been addressed. 1241 cases have no county]
# unclaimed_county_null_df.to_csv('unclaimed_county_nulls.csv', encoding='Windows-1252')

In [105]:
# check unclaimed_df
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS,State_County,County_FIPS
0,UCP68231,4/15/2020,Sanders,Stephen,Male,White / Caucasian,Joliet,Will,Illinois,4/16/2020,17,17_Will,17197.0
1,UCP68248,4/13/2020,Brookshire,Renee,Female,White / Caucasian,Joliet,Will,Illinois,4/16/2020,17,17_Will,17197.0
2,UCP68242,4/9/2020,Smith,Victoria,Female,White / Caucasian,Manhattan,New York,New York,4/16/2020,36,36_New York,36061.0
3,UCP68244,4/9/2020,Bellamy,Eldred,Male,Black / African American,Brooklyn,Kings,New York,4/16/2020,36,36_Kings,36047.0
4,UCP68228,4/9/2020,Rodriguez,Elido,Male,Hispanic / Latino,Manhattan,New York,New York,4/15/2020,36,36_New York,36061.0


##### Replace null state values with 'None' and null county values with 'None', then re-write county_FIPS column using county_centroids_v2_dataframe

In [106]:
## Check if any rows with null value for 'State'
unc_state_test_df = unclaimed_df.loc[unclaimed_df['State'].isnull()]
# len(unc_state_test_df)
unc_state_test_df

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS,State_County,County_FIPS
3380,UCP5339,12/3/2016,Williams,Barbara,Female,Asian,NaN,NaN,NaN,5/3/2018,99,NaN,NaN
4774,UCP2489,3/26/2015,Lupien,Daryl,Male,Uncertain,NaN,NaN,NaN,8/18/2016,99,NaN,NaN
5023,UCP4001,11/2/2014,Brown,David,Male,White / Caucasian,NaN,NaN,NaN,5/11/2017,99,NaN,NaN
5883,UCP3962,3/22/2010,Gonzalez,David,Male,Hispanic / Latino,NaN,NaN,NaN,5/11/2017,99,NaN,NaN
5950,UCP3937,12/7/2009,Togovnick,Bernice,Female,White / Caucasian,NaN,NaN,NaN,5/11/2017,99,NaN,NaN
6183,UCP3901,1/21/2009,Carney,John,Male,White / Caucasian,NaN,NaN,NaN,5/11/2017,99,NaN,NaN
6341,UCP3689,6/5/2008,Guzman,Ralph,Male,Hispanic / Latino,NaN,NaN,NaN,5/2/2017,99,NaN,NaN
6560,UCP3824,7/2/2007,Vasquez,Rafael,Male,Hispanic / Latino,NaN,NaN,NaN,5/4/2017,99,NaN,NaN
6698,UCP3672,9/22/2006,Barajas,Epifanio,Male,Hispanic / Latino,NaN,NaN,NaN,5/2/2017,99,NaN,NaN
6719,UCP3668,8/5/2006,Huff,Paul,Male,White / Caucasian,NaN,NaN,NaN,5/2/2017,99,NaN,NaN


In [107]:
# # change NaN State name (for cases with no city or county or state) to 'None'
unclaimed_df['State'] = unclaimed_df['State'].fillna('None')
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS,State_County,County_FIPS
0,UCP68231,4/15/2020,Sanders,Stephen,Male,White / Caucasian,Joliet,Will,Illinois,4/16/2020,17,17_Will,17197.0
1,UCP68248,4/13/2020,Brookshire,Renee,Female,White / Caucasian,Joliet,Will,Illinois,4/16/2020,17,17_Will,17197.0
2,UCP68242,4/9/2020,Smith,Victoria,Female,White / Caucasian,Manhattan,New York,New York,4/16/2020,36,36_New York,36061.0
3,UCP68244,4/9/2020,Bellamy,Eldred,Male,Black / African American,Brooklyn,Kings,New York,4/16/2020,36,36_Kings,36047.0
4,UCP68228,4/9/2020,Rodriguez,Elido,Male,Hispanic / Latino,Manhattan,New York,New York,4/15/2020,36,36_New York,36061.0


In [108]:
## Double check re-assigned values
unc_state_test_2_df = unclaimed_df.loc[unclaimed_df['State']=='None']
unc_state_test_2_df
# len(unc_state_test_2_df)

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS,State_County,County_FIPS
3380,UCP5339,12/3/2016,Williams,Barbara,Female,Asian,NaN,NaN,None,5/3/2018,99,NaN,NaN
4774,UCP2489,3/26/2015,Lupien,Daryl,Male,Uncertain,NaN,NaN,None,8/18/2016,99,NaN,NaN
5023,UCP4001,11/2/2014,Brown,David,Male,White / Caucasian,NaN,NaN,None,5/11/2017,99,NaN,NaN
5883,UCP3962,3/22/2010,Gonzalez,David,Male,Hispanic / Latino,NaN,NaN,None,5/11/2017,99,NaN,NaN
5950,UCP3937,12/7/2009,Togovnick,Bernice,Female,White / Caucasian,NaN,NaN,None,5/11/2017,99,NaN,NaN
6183,UCP3901,1/21/2009,Carney,John,Male,White / Caucasian,NaN,NaN,None,5/11/2017,99,NaN,NaN
6341,UCP3689,6/5/2008,Guzman,Ralph,Male,Hispanic / Latino,NaN,NaN,None,5/2/2017,99,NaN,NaN
6560,UCP3824,7/2/2007,Vasquez,Rafael,Male,Hispanic / Latino,NaN,NaN,None,5/4/2017,99,NaN,NaN
6698,UCP3672,9/22/2006,Barajas,Epifanio,Male,Hispanic / Latino,NaN,NaN,None,5/2/2017,99,NaN,NaN
6719,UCP3668,8/5/2006,Huff,Paul,Male,White / Caucasian,NaN,NaN,None,5/2/2017,99,NaN,NaN


In [109]:
## Check if any rows with null value for 'City' and County'
unc_county_test_df = unclaimed_df.loc[unclaimed_df['County'].isnull()]
# len(unc_county_test_df)
unc_county_test_df

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS,State_County,County_FIPS
1996,UCP5438,4/30/2018,Flores,Victor,Male,Black / African American,NaN,NaN,New York,5/14/2019,36,NaN,NaN
2011,UCP5440,4/22/2018,Browne,Edward,Male,Hispanic / Latino,NaN,NaN,New York,5/27/2019,36,NaN,NaN
2028,UCP5297,4/15/2018,Davis,Thelma,Female,Black / African American,NaN,NaN,New York,5/20/2019,36,NaN,NaN
2031,UCP5296,4/14/2018,Fasulo,Timothy,Male,White / Caucasian,NaN,NaN,New York,4/25/2019,36,NaN,NaN
2033,UCP5439,4/14/2018,Reyes,Nivia,Female,Other,NaN,NaN,New York,5/27/2019,36,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7286,UCP990,11/6/1997,Pope,Leon,Male,White / Caucasian,NaN,NaN,Washington,4/14/2020,53,NaN,NaN
7339,UCP1999,10/22/1991,Robles,Salvador,Male,Other,NaN,NaN,Tennessee,1/10/2020,47,NaN,NaN
7344,UCP4522,10/18/1990,Way,Kenneth,Male,Black / African American,NaN,NaN,New York,5/11/2019,36,NaN,NaN
7372,UCP1548,6/17/1973,Seah,Tee kee,Male,Asian,NaN,NaN,Nevada,5/28/2019,32,NaN,NaN


In [110]:
# # change NaN County name (for cases with no city or county) to 'None'
unclaimed_df['County'] = unclaimed_df['County'].fillna('None')
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS,State_County,County_FIPS
0,UCP68231,4/15/2020,Sanders,Stephen,Male,White / Caucasian,Joliet,Will,Illinois,4/16/2020,17,17_Will,17197.0
1,UCP68248,4/13/2020,Brookshire,Renee,Female,White / Caucasian,Joliet,Will,Illinois,4/16/2020,17,17_Will,17197.0
2,UCP68242,4/9/2020,Smith,Victoria,Female,White / Caucasian,Manhattan,New York,New York,4/16/2020,36,36_New York,36061.0
3,UCP68244,4/9/2020,Bellamy,Eldred,Male,Black / African American,Brooklyn,Kings,New York,4/16/2020,36,36_Kings,36047.0
4,UCP68228,4/9/2020,Rodriguez,Elido,Male,Hispanic / Latino,Manhattan,New York,New York,4/15/2020,36,36_New York,36061.0


In [111]:
## Double check re-assigned values
unc_county_test_2_df = unclaimed_df.loc[unclaimed_df['County']=='None']
unc_county_test_2_df
# len(unc_county_test_2_df)

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS,State_County,County_FIPS
1996,UCP5438,4/30/2018,Flores,Victor,Male,Black / African American,NaN,None,New York,5/14/2019,36,NaN,NaN
2011,UCP5440,4/22/2018,Browne,Edward,Male,Hispanic / Latino,NaN,None,New York,5/27/2019,36,NaN,NaN
2028,UCP5297,4/15/2018,Davis,Thelma,Female,Black / African American,NaN,None,New York,5/20/2019,36,NaN,NaN
2031,UCP5296,4/14/2018,Fasulo,Timothy,Male,White / Caucasian,NaN,None,New York,4/25/2019,36,NaN,NaN
2033,UCP5439,4/14/2018,Reyes,Nivia,Female,Other,NaN,None,New York,5/27/2019,36,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7286,UCP990,11/6/1997,Pope,Leon,Male,White / Caucasian,NaN,None,Washington,4/14/2020,53,NaN,NaN
7339,UCP1999,10/22/1991,Robles,Salvador,Male,Other,NaN,None,Tennessee,1/10/2020,47,NaN,NaN
7344,UCP4522,10/18/1990,Way,Kenneth,Male,Black / African American,NaN,None,New York,5/11/2019,36,NaN,NaN
7372,UCP1548,6/17/1973,Seah,Tee kee,Male,Asian,NaN,None,Nevada,5/28/2019,32,NaN,NaN


In [112]:
# Re-Add column with compound field key
unclaimed_df['State_County'] = unclaimed_df['State_FIPS'].astype(str) + "_" + unclaimed_df['County']
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS,State_County,County_FIPS
0,UCP68231,4/15/2020,Sanders,Stephen,Male,White / Caucasian,Joliet,Will,Illinois,4/16/2020,17,17_Will,17197.0
1,UCP68248,4/13/2020,Brookshire,Renee,Female,White / Caucasian,Joliet,Will,Illinois,4/16/2020,17,17_Will,17197.0
2,UCP68242,4/9/2020,Smith,Victoria,Female,White / Caucasian,Manhattan,New York,New York,4/16/2020,36,36_New York,36061.0
3,UCP68244,4/9/2020,Bellamy,Eldred,Male,Black / African American,Brooklyn,Kings,New York,4/16/2020,36,36_Kings,36047.0
4,UCP68228,4/9/2020,Rodriguez,Elido,Male,Hispanic / Latino,Manhattan,New York,New York,4/15/2020,36,36_New York,36061.0


In [113]:
## Double check re-assigned values
unc_county_test_3_df = unclaimed_df.loc[unclaimed_df['County']=='None']
unc_county_test_3_df
# len(unc_county_test_3_df)

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS,State_County,County_FIPS
1996,UCP5438,4/30/2018,Flores,Victor,Male,Black / African American,NaN,None,New York,5/14/2019,36,36_None,NaN
2011,UCP5440,4/22/2018,Browne,Edward,Male,Hispanic / Latino,NaN,None,New York,5/27/2019,36,36_None,NaN
2028,UCP5297,4/15/2018,Davis,Thelma,Female,Black / African American,NaN,None,New York,5/20/2019,36,36_None,NaN
2031,UCP5296,4/14/2018,Fasulo,Timothy,Male,White / Caucasian,NaN,None,New York,4/25/2019,36,36_None,NaN
2033,UCP5439,4/14/2018,Reyes,Nivia,Female,Other,NaN,None,New York,5/27/2019,36,36_None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7286,UCP990,11/6/1997,Pope,Leon,Male,White / Caucasian,NaN,None,Washington,4/14/2020,53,53_None,NaN
7339,UCP1999,10/22/1991,Robles,Salvador,Male,Other,NaN,None,Tennessee,1/10/2020,47,47_None,NaN
7344,UCP4522,10/18/1990,Way,Kenneth,Male,Black / African American,NaN,None,New York,5/11/2019,36,36_None,NaN
7372,UCP1548,6/17/1973,Seah,Tee kee,Male,Asian,NaN,None,Nevada,5/28/2019,32,32_None,NaN


In [114]:
# Re-add County FIPS column to dataframe, using the State_County name field as a key in the county_v2_dict, to pull the correct County FIPS code for each row
unclaimed_df['County_FIPS'] = unclaimed_df['State_County'].map(county_v2_dict)
# Check unique values in new dataframe field
unclaimed_df['County_FIPS'].unique()

array([17197, 36061, 36047, 36005, 36081, 25025,  6073, 17063, 36085,
       53053, 15003, 32031, 25021, 25027, 16001, 25009, 25017, 25003,
       25013, 25023, 32027, 25015,  6047, 48201, 49035, 47093, 53075,
       53027, 11001, 25005, 47157, 33005, 35031, 53049, 53077, 40143,
       42029, 47155, 47001, 39099, 29019,  6075,  5143,  6093, 13121,
        1073, 34025, 34023, 25001,  4019,  4013,  6023,  6099, 34001,
       12087, 47035,  6037, 40121, 53061, 55025,  6071,  9003, 48141,
       48339,  9009, 42101, 12071, 53033, 17043, 34021,  9011, 48215,
        8087, 12051, 26163,  9001, 47009, 17031, 22069,  9005, 32005,
        9007, 47065, 34013, 36999,  4007, 47013, 18067, 42001,  6059,
       19155,  6083, 26025, 40101, 40041, 42091, 55057, 17999, 48167,
       19153,  6025, 40999, 42999, 26121, 40145, 25011,  9015,  1115,
        5035, 53035, 53073, 41051, 16027, 23005, 47179, 48469, 40135,
       99999, 34029, 40097, 16013, 29051, 48229, 40115, 45007, 12075,
       35059, 53001,

In [115]:
# check null values to make sure none are left [Note - all gone!]
unclaimed_county_null_v2_df = unclaimed_df.loc[unclaimed_df['County_FIPS'].isnull()]
# unclaimed_county_null_v2_df.shape
unclaimed_county_null_v2_df

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS,State_County,County_FIPS


In [116]:
## Double check re-assigned values
unc_county_test_4_df = unclaimed_df.loc[unclaimed_df['County']=='None']
unc_county_test_4_df
# len(unc_county_test_4_df)

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS,State_County,County_FIPS
1996,UCP5438,4/30/2018,Flores,Victor,Male,Black / African American,NaN,None,New York,5/14/2019,36,36_None,36999
2011,UCP5440,4/22/2018,Browne,Edward,Male,Hispanic / Latino,NaN,None,New York,5/27/2019,36,36_None,36999
2028,UCP5297,4/15/2018,Davis,Thelma,Female,Black / African American,NaN,None,New York,5/20/2019,36,36_None,36999
2031,UCP5296,4/14/2018,Fasulo,Timothy,Male,White / Caucasian,NaN,None,New York,4/25/2019,36,36_None,36999
2033,UCP5439,4/14/2018,Reyes,Nivia,Female,Other,NaN,None,New York,5/27/2019,36,36_None,36999
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7286,UCP990,11/6/1997,Pope,Leon,Male,White / Caucasian,NaN,None,Washington,4/14/2020,53,53_None,53999
7339,UCP1999,10/22/1991,Robles,Salvador,Male,Other,NaN,None,Tennessee,1/10/2020,47,47_None,47999
7344,UCP4522,10/18/1990,Way,Kenneth,Male,Black / African American,NaN,None,New York,5/11/2019,36,36_None,36999
7372,UCP1548,6/17/1973,Seah,Tee kee,Male,Asian,NaN,None,Nevada,5/28/2019,32,32_None,32999


#### Add in County FIPS column to unidentified dataframe

##### To identify null values, add in County FIPS column to unidentified dataframe using county_centroids_df

In [117]:
# check unidentified df
unidentified_df.head()

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS
0,UP51903,3/30/1986,0.0,120.0,Marbury,Autauga,Alabama,Female,White / Caucasian,1/28/2020,1
1,UP5000,9/29/2007,40.0,60.0,Prattville,Autauga,Alabama,Female,Black / African American,8/12/2019,1
2,UP6519,1/6/2006,45.0,59.0,Elberta,Baldwin,Alabama,Male,White / Caucasian,10/24/2018,1
3,UP55409,3/26/2000,0.0,120.0,NaN,Baldwin,Alabama,Male,"White / Caucasian, Asian, American Indian / Al...",2/13/2020,1
4,UP13483,3/27/1961,14.0,17.0,NaN,Bibb,Alabama,Male,White / Caucasian,3/25/2019,1


In [118]:
# Add column with compound field key
unidentified_df['State_County'] = unidentified_df['State_FIPS'].astype(str) + "_" + unidentified_df['County']
unidentified_df.head()

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County
0,UP51903,3/30/1986,0.0,120.0,Marbury,Autauga,Alabama,Female,White / Caucasian,1/28/2020,1,1_Autauga
1,UP5000,9/29/2007,40.0,60.0,Prattville,Autauga,Alabama,Female,Black / African American,8/12/2019,1,1_Autauga
2,UP6519,1/6/2006,45.0,59.0,Elberta,Baldwin,Alabama,Male,White / Caucasian,10/24/2018,1,1_Baldwin
3,UP55409,3/26/2000,0.0,120.0,NaN,Baldwin,Alabama,Male,"White / Caucasian, Asian, American Indian / Al...",2/13/2020,1,1_Baldwin
4,UP13483,3/27/1961,14.0,17.0,NaN,Bibb,Alabama,Male,White / Caucasian,3/25/2019,1,1_Bibb


In [119]:
# Add new column to dataframe, using the state name field as a key in the state_dict, to pull the correct FIPS code for each row
unidentified_df['County_FIPS'] = unidentified_df['State_County'].map(county_dict)
# Check unique values in new dataframe field
unidentified_df['County_FIPS'].unique()

array([ 1001.,  1003.,  1007., ..., 56021., 56033., 56037.])

In [120]:
# check null values
unidentified_county_null_df = unidentified_df.loc[unidentified_df['County_FIPS'].isnull()]
# unidentified_county_null_df.shape
# len(unidentified_county_null_df)
unidentified_county_null_df

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS
55,UP14028,12/21/1991,18.0,25.0,NaN,NaN,Alabama,Male,White / Caucasian,3/31/2017,1,NaN,NaN
109,UP13395,9/20/2004,0.0,120.0,NaN,NaN,Alaska,Male,Uncertain,4/14/2020,2,NaN,NaN
110,UP7281,4/30/2001,20.0,30.0,NaN,NaN,Alaska,Male,White / Caucasian,4/14/2020,2,NaN,NaN
111,UP13575,9/11/1997,0.0,120.0,NaN,NaN,Alaska,Male,Uncertain,7/2/2018,2,NaN,NaN
6392,UP4846,4/14/1988,37.0,52.0,NaN,NaN,Indiana,Female,White / Caucasian,6/3/2017,18,NaN,NaN
7676,UP5018,4/19/1996,49.0,59.0,NaN,NaN,Missouri,Female,Black / African American,10/23/2018,29,NaN,NaN
8536,UP17664,1/29/2018,0.0,120.0,NaN,NaN,New Mexico,Male,Uncertain,1/21/2020,35,NaN,NaN
8540,UP16690,5/3/2017,0.0,120.0,NaN,NaN,New Mexico,Unsure,Uncertain,2/8/2020,35,NaN,NaN
8541,UP5578,3/26/2003,50.0,60.0,NaN,NaN,New Mexico,Female,Black / African American,10/24/2018,35,NaN,NaN
10966,UP56112,10/10/2017,0.0,120.0,NaN,NaN,Puerto Rico,Male,Uncertain,2/10/2020,72,NaN,NaN


In [121]:
# As needed, export nulls to address [NOTE - all have been addressed - 28 remain with no city or county]
# unidentified_county_null_df.to_csv('unidentified_county_nulls.csv', encoding='Windows-1252')

In [122]:
# check unclaimed_df
unidentified_df.head()

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS
0,UP51903,3/30/1986,0.0,120.0,Marbury,Autauga,Alabama,Female,White / Caucasian,1/28/2020,1,1_Autauga,1001.0
1,UP5000,9/29/2007,40.0,60.0,Prattville,Autauga,Alabama,Female,Black / African American,8/12/2019,1,1_Autauga,1001.0
2,UP6519,1/6/2006,45.0,59.0,Elberta,Baldwin,Alabama,Male,White / Caucasian,10/24/2018,1,1_Baldwin,1003.0
3,UP55409,3/26/2000,0.0,120.0,NaN,Baldwin,Alabama,Male,"White / Caucasian, Asian, American Indian / Al...",2/13/2020,1,1_Baldwin,1003.0
4,UP13483,3/27/1961,14.0,17.0,NaN,Bibb,Alabama,Male,White / Caucasian,3/25/2019,1,1_Bibb,1007.0


##### Replace null state values with 'None' and null county values with 'None', then re-write county_FIPS column using county_centroids_v2_dataframe

In [123]:
## Check if any rows with null value for 'State'
uni_state_test_df = unidentified_df.loc[unidentified_df['State'].isnull()]
# len(uni_state_test_df)
uni_state_test_df

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS


In [124]:
## Check if any rows with null value for 'City' and County'
uni_county_test_df = unidentified_df.loc[unidentified_df['County'].isnull()]
# len(uni_county_test_df)
uni_county_test_df

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS
55,UP14028,12/21/1991,18.0,25.0,NaN,NaN,Alabama,Male,White / Caucasian,3/31/2017,1,NaN,NaN
109,UP13395,9/20/2004,0.0,120.0,NaN,NaN,Alaska,Male,Uncertain,4/14/2020,2,NaN,NaN
110,UP7281,4/30/2001,20.0,30.0,NaN,NaN,Alaska,Male,White / Caucasian,4/14/2020,2,NaN,NaN
111,UP13575,9/11/1997,0.0,120.0,NaN,NaN,Alaska,Male,Uncertain,7/2/2018,2,NaN,NaN
6392,UP4846,4/14/1988,37.0,52.0,NaN,NaN,Indiana,Female,White / Caucasian,6/3/2017,18,NaN,NaN
7676,UP5018,4/19/1996,49.0,59.0,NaN,NaN,Missouri,Female,Black / African American,10/23/2018,29,NaN,NaN
8536,UP17664,1/29/2018,0.0,120.0,NaN,NaN,New Mexico,Male,Uncertain,1/21/2020,35,NaN,NaN
8540,UP16690,5/3/2017,0.0,120.0,NaN,NaN,New Mexico,Unsure,Uncertain,2/8/2020,35,NaN,NaN
8541,UP5578,3/26/2003,50.0,60.0,NaN,NaN,New Mexico,Female,Black / African American,10/24/2018,35,NaN,NaN
10966,UP56112,10/10/2017,0.0,120.0,NaN,NaN,Puerto Rico,Male,Uncertain,2/10/2020,72,NaN,NaN


In [125]:
# # change NaN County name (for cases with no city or county) to 'None'
unidentified_df['County'] = unidentified_df['County'].fillna('None')
unidentified_df.head()

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS
0,UP51903,3/30/1986,0.0,120.0,Marbury,Autauga,Alabama,Female,White / Caucasian,1/28/2020,1,1_Autauga,1001.0
1,UP5000,9/29/2007,40.0,60.0,Prattville,Autauga,Alabama,Female,Black / African American,8/12/2019,1,1_Autauga,1001.0
2,UP6519,1/6/2006,45.0,59.0,Elberta,Baldwin,Alabama,Male,White / Caucasian,10/24/2018,1,1_Baldwin,1003.0
3,UP55409,3/26/2000,0.0,120.0,NaN,Baldwin,Alabama,Male,"White / Caucasian, Asian, American Indian / Al...",2/13/2020,1,1_Baldwin,1003.0
4,UP13483,3/27/1961,14.0,17.0,NaN,Bibb,Alabama,Male,White / Caucasian,3/25/2019,1,1_Bibb,1007.0


In [126]:
## Double check re-assigned values
uni_county_test_2_df = unidentified_df.loc[unidentified_df['County']=='None']
uni_county_test_2_df
# len(uni_county_test_2_df)

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS
55,UP14028,12/21/1991,18.0,25.0,NaN,None,Alabama,Male,White / Caucasian,3/31/2017,1,NaN,NaN
109,UP13395,9/20/2004,0.0,120.0,NaN,None,Alaska,Male,Uncertain,4/14/2020,2,NaN,NaN
110,UP7281,4/30/2001,20.0,30.0,NaN,None,Alaska,Male,White / Caucasian,4/14/2020,2,NaN,NaN
111,UP13575,9/11/1997,0.0,120.0,NaN,None,Alaska,Male,Uncertain,7/2/2018,2,NaN,NaN
6392,UP4846,4/14/1988,37.0,52.0,NaN,None,Indiana,Female,White / Caucasian,6/3/2017,18,NaN,NaN
7676,UP5018,4/19/1996,49.0,59.0,NaN,None,Missouri,Female,Black / African American,10/23/2018,29,NaN,NaN
8536,UP17664,1/29/2018,0.0,120.0,NaN,None,New Mexico,Male,Uncertain,1/21/2020,35,NaN,NaN
8540,UP16690,5/3/2017,0.0,120.0,NaN,None,New Mexico,Unsure,Uncertain,2/8/2020,35,NaN,NaN
8541,UP5578,3/26/2003,50.0,60.0,NaN,None,New Mexico,Female,Black / African American,10/24/2018,35,NaN,NaN
10966,UP56112,10/10/2017,0.0,120.0,NaN,None,Puerto Rico,Male,Uncertain,2/10/2020,72,NaN,NaN


In [127]:
# Re-Add column with compound field key
unidentified_df['State_County'] = unidentified_df['State_FIPS'].astype(str) + "_" + unidentified_df['County']
unidentified_df.head()

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS
0,UP51903,3/30/1986,0.0,120.0,Marbury,Autauga,Alabama,Female,White / Caucasian,1/28/2020,1,1_Autauga,1001.0
1,UP5000,9/29/2007,40.0,60.0,Prattville,Autauga,Alabama,Female,Black / African American,8/12/2019,1,1_Autauga,1001.0
2,UP6519,1/6/2006,45.0,59.0,Elberta,Baldwin,Alabama,Male,White / Caucasian,10/24/2018,1,1_Baldwin,1003.0
3,UP55409,3/26/2000,0.0,120.0,NaN,Baldwin,Alabama,Male,"White / Caucasian, Asian, American Indian / Al...",2/13/2020,1,1_Baldwin,1003.0
4,UP13483,3/27/1961,14.0,17.0,NaN,Bibb,Alabama,Male,White / Caucasian,3/25/2019,1,1_Bibb,1007.0


In [128]:
## Double check re-assigned values
uni_county_test_3_df = unidentified_df.loc[unidentified_df['County']=='None']
uni_county_test_3_df
# len(uni_county_test_3_df)

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS
55,UP14028,12/21/1991,18.0,25.0,NaN,None,Alabama,Male,White / Caucasian,3/31/2017,1,1_None,NaN
109,UP13395,9/20/2004,0.0,120.0,NaN,None,Alaska,Male,Uncertain,4/14/2020,2,2_None,NaN
110,UP7281,4/30/2001,20.0,30.0,NaN,None,Alaska,Male,White / Caucasian,4/14/2020,2,2_None,NaN
111,UP13575,9/11/1997,0.0,120.0,NaN,None,Alaska,Male,Uncertain,7/2/2018,2,2_None,NaN
6392,UP4846,4/14/1988,37.0,52.0,NaN,None,Indiana,Female,White / Caucasian,6/3/2017,18,18_None,NaN
7676,UP5018,4/19/1996,49.0,59.0,NaN,None,Missouri,Female,Black / African American,10/23/2018,29,29_None,NaN
8536,UP17664,1/29/2018,0.0,120.0,NaN,None,New Mexico,Male,Uncertain,1/21/2020,35,35_None,NaN
8540,UP16690,5/3/2017,0.0,120.0,NaN,None,New Mexico,Unsure,Uncertain,2/8/2020,35,35_None,NaN
8541,UP5578,3/26/2003,50.0,60.0,NaN,None,New Mexico,Female,Black / African American,10/24/2018,35,35_None,NaN
10966,UP56112,10/10/2017,0.0,120.0,NaN,None,Puerto Rico,Male,Uncertain,2/10/2020,72,72_None,NaN


In [129]:
# Re-add County FIPS column to dataframe, using the State_County name field as a key in the county_v2_dict, to pull the correct County FIPS code for each row
unidentified_df['County_FIPS'] = unidentified_df['State_County'].map(county_v2_dict)
# Check unique values in new dataframe field
unidentified_df['County_FIPS'].unique()

array([ 1001,  1003,  1007, ..., 56021, 56033, 56037], dtype=int64)

In [130]:
# check null values to make sure none are left [Note - all gone!]
unidentified_county_null_v2_df = unidentified_df.loc[unidentified_df['County_FIPS'].isnull()]
# unidentified_county_null_v2_df.shape
unidentified_county_null_v2_df

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS


In [131]:
## Double check re-assigned values
uni_county_test_4_df = unidentified_df.loc[unidentified_df['County']=='None']
uni_county_test_4_df
# len(uni_county_test_4_df)

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS
55,UP14028,12/21/1991,18.0,25.0,NaN,None,Alabama,Male,White / Caucasian,3/31/2017,1,1_None,1999
109,UP13395,9/20/2004,0.0,120.0,NaN,None,Alaska,Male,Uncertain,4/14/2020,2,2_None,2999
110,UP7281,4/30/2001,20.0,30.0,NaN,None,Alaska,Male,White / Caucasian,4/14/2020,2,2_None,2999
111,UP13575,9/11/1997,0.0,120.0,NaN,None,Alaska,Male,Uncertain,7/2/2018,2,2_None,2999
6392,UP4846,4/14/1988,37.0,52.0,NaN,None,Indiana,Female,White / Caucasian,6/3/2017,18,18_None,18999
7676,UP5018,4/19/1996,49.0,59.0,NaN,None,Missouri,Female,Black / African American,10/23/2018,29,29_None,29999
8536,UP17664,1/29/2018,0.0,120.0,NaN,None,New Mexico,Male,Uncertain,1/21/2020,35,35_None,35999
8540,UP16690,5/3/2017,0.0,120.0,NaN,None,New Mexico,Unsure,Uncertain,2/8/2020,35,35_None,35999
8541,UP5578,3/26/2003,50.0,60.0,NaN,None,New Mexico,Female,Black / African American,10/24/2018,35,35_None,35999
10966,UP56112,10/10/2017,0.0,120.0,NaN,None,Puerto Rico,Male,Uncertain,2/10/2020,72,72_None,72999


# Part 4: Prep data for city-level JSONs for all 3 databases

#### Clean up city dataframe

In [132]:
city_df.head()

,OBJECTID,FEATURE_ID,FEATURE_NA,FEATURE_CL,STATE_ALPH,STATE_NUME,COUNTY_NAM,COUNTY_NUM,PRIMARY_LA,PRIM_LONG_,...,PRIM_LON_1,SOURCE_LAT,SOURCE_LON,SOURCE_L_1,SOURCE_L_2,ELEV_IN_M,ELEV_IN_FT,MAP_NAME,DATE_CREAT,DATE_EDITE
0,1,479.0,Adamana,Populated Place,AZ,4.0,Apache,1.0,345836N,1094920W,...,-109.822338,,,0.0,0.0,1617.0,5305.0,Adamana,02/08/1980,
1,2,492.0,Adobe,Populated Place,AZ,4.0,Maricopa,13.0,334121N,1120722W,...,-112.122653,,,0.0,0.0,425.0,1394.0,Union Hills,02/08/1980,
2,3,511.0,Agua Fria,Populated Place,AZ,4.0,Maricopa,13.0,333620N,1121853W,...,-112.314604,,,0.0,0.0,342.0,1122.0,El Mirage,02/08/1980,
3,4,516.0,Aguila,Populated Place,AZ,4.0,Maricopa,13.0,335634N,1131027W,...,-113.174081,,,0.0,0.0,659.0,2162.0,Aguila,02/08/1980,01/29/2011
4,5,538.0,Ajo,Populated Place,AZ,4.0,Pima,19.0,322218N,1125139W,...,-112.860710,,,0.0,0.0,536.0,1759.0,Ajo South,02/08/1980,03/21/2008


In [133]:
# Drop columns that aren't needed
city_df = city_df.drop(columns=['FEATURE_CL','PRIMARY_LA', 'PRIM_LONG_', 'SOURCE_LAT', 'SOURCE_LON', 'SOURCE_L_1', 'SOURCE_L_2', 'ELEV_IN_M', 'ELEV_IN_FT', 'MAP_NAME', 'DATE_CREAT', 'DATE_EDITE'])

In [134]:
city_df.head()

,OBJECTID,FEATURE_ID,FEATURE_NA,STATE_ALPH,STATE_NUME,COUNTY_NAM,COUNTY_NUM,PRIM_LAT_D,PRIM_LON_1
0,1,479.0,Adamana,AZ,4.0,Apache,1.0,34.976692,-109.822338
1,2,492.0,Adobe,AZ,4.0,Maricopa,13.0,33.689205,-112.122653
2,3,511.0,Agua Fria,AZ,4.0,Maricopa,13.0,33.605595,-112.314604
3,4,516.0,Aguila,AZ,4.0,Maricopa,13.0,33.942807,-113.174081
4,5,538.0,Ajo,AZ,4.0,Pima,19.0,32.371725,-112.860710


In [135]:
# Rename columns
city_df.rename(columns={'FEATURE_NA':'CITY_NAME', 'STATE_ALPH':'STUSPS', 'STATE_NUME':'STATEFP', 'COUNTY_NAM':'COUNTY_NAME', 'PRIM_LAT_D': 'Lat_dd', 'PRIM_LON_1': 'Lon_dd'}, inplace=True)

In [136]:
city_df.head()

,OBJECTID,FEATURE_ID,CITY_NAME,STUSPS,STATEFP,COUNTY_NAME,COUNTY_NUM,Lat_dd,Lon_dd
0,1,479.0,Adamana,AZ,4.0,Apache,1.0,34.976692,-109.822338
1,2,492.0,Adobe,AZ,4.0,Maricopa,13.0,33.689205,-112.122653
2,3,511.0,Agua Fria,AZ,4.0,Maricopa,13.0,33.605595,-112.314604
3,4,516.0,Aguila,AZ,4.0,Maricopa,13.0,33.942807,-113.174081
4,5,538.0,Ajo,AZ,4.0,Pima,19.0,32.371725,-112.860710


In [137]:
# Make state FIPS int
city_df['STATEFP'] = city_df['STATEFP'].astype(int)
city_df.head()

,OBJECTID,FEATURE_ID,CITY_NAME,STUSPS,STATEFP,COUNTY_NAME,COUNTY_NUM,Lat_dd,Lon_dd
0,1,479.0,Adamana,AZ,4,Apache,1.0,34.976692,-109.822338
1,2,492.0,Adobe,AZ,4,Maricopa,13.0,33.689205,-112.122653
2,3,511.0,Agua Fria,AZ,4,Maricopa,13.0,33.605595,-112.314604
3,4,516.0,Aguila,AZ,4,Maricopa,13.0,33.942807,-113.174081
4,5,538.0,Ajo,AZ,4,Pima,19.0,32.371725,-112.860710


In [138]:
# Make couty FIPS int
city_df['COUNTY_NUM'] = city_df['COUNTY_NUM'].astype(int)
city_df.head()

,OBJECTID,FEATURE_ID,CITY_NAME,STUSPS,STATEFP,COUNTY_NAME,COUNTY_NUM,Lat_dd,Lon_dd
0,1,479.0,Adamana,AZ,4,Apache,1,34.976692,-109.822338
1,2,492.0,Adobe,AZ,4,Maricopa,13,33.689205,-112.122653
2,3,511.0,Agua Fria,AZ,4,Maricopa,13,33.605595,-112.314604
3,4,516.0,Aguila,AZ,4,Maricopa,13,33.942807,-113.174081
4,5,538.0,Ajo,AZ,4,Pima,19,32.371725,-112.860710


In [139]:
# reformat county num with leading zeros
city_df['COUNTY_NUM']=city_df['COUNTY_NUM'].apply('{:0>3}'.format)
city_df.head()

,OBJECTID,FEATURE_ID,CITY_NAME,STUSPS,STATEFP,COUNTY_NAME,COUNTY_NUM,Lat_dd,Lon_dd
0,1,479.0,Adamana,AZ,4,Apache,001,34.976692,-109.822338
1,2,492.0,Adobe,AZ,4,Maricopa,013,33.689205,-112.122653
2,3,511.0,Agua Fria,AZ,4,Maricopa,013,33.605595,-112.314604
3,4,516.0,Aguila,AZ,4,Maricopa,013,33.942807,-113.174081
4,5,538.0,Ajo,AZ,4,Pima,019,32.371725,-112.860710


In [140]:
# create county FIPS field
city_df['COUNTY_FIPS'] = city_df['STATEFP'].astype(str) + city_df['COUNTY_NUM'].astype(str)
city_df.head(5000)

,OBJECTID,FEATURE_ID,CITY_NAME,STUSPS,STATEFP,COUNTY_NAME,COUNTY_NUM,Lat_dd,Lon_dd,COUNTY_FIPS
0,1,479.0,Adamana,AZ,4,Apache,001,34.976692,-109.822338,4001
1,2,492.0,Adobe,AZ,4,Maricopa,013,33.689205,-112.122653,4013
2,3,511.0,Agua Fria,AZ,4,Maricopa,013,33.605595,-112.314604,4013
3,4,516.0,Aguila,AZ,4,Maricopa,013,33.942807,-113.174081,4013
4,5,538.0,Ajo,AZ,4,Pima,019,32.371725,-112.860710,4019
...,...,...,...,...,...,...,...,...,...,...
4995,4996,70588.0,Frog Town,AR,5,Sebastian,131,35.039820,-94.354382,5131
4996,4997,70624.0,Ben Hur,AR,5,Newton,101,35.730357,-92.972950,5101
4997,4998,70626.0,Bend,AR,5,Johnson,071,35.354251,-93.387404,5071
4998,4999,70650.0,Bidville,AR,5,Crawford,033,35.739803,-93.981865,5033


In [141]:
# Add new column to dataframe, using the state fips field as a key in the state_dict, to pull the correct FIPS code for each row
city_df['STATE_NAME'] = city_df['STATEFP'].map(state_FIPS_dict)
city_df.head()

,OBJECTID,FEATURE_ID,CITY_NAME,STUSPS,STATEFP,COUNTY_NAME,COUNTY_NUM,Lat_dd,Lon_dd,COUNTY_FIPS,STATE_NAME
0,1,479.0,Adamana,AZ,4,Apache,001,34.976692,-109.822338,4001,Arizona
1,2,492.0,Adobe,AZ,4,Maricopa,013,33.689205,-112.122653,4013,Arizona
2,3,511.0,Agua Fria,AZ,4,Maricopa,013,33.605595,-112.314604,4013,Arizona
3,4,516.0,Aguila,AZ,4,Maricopa,013,33.942807,-113.174081,4013,Arizona
4,5,538.0,Ajo,AZ,4,Pima,019,32.371725,-112.860710,4019,Arizona


In [142]:
city_df['CITY_STRING'] = city_df['CITY_NAME']
city_df.head()

,OBJECTID,FEATURE_ID,CITY_NAME,STUSPS,STATEFP,COUNTY_NAME,COUNTY_NUM,Lat_dd,Lon_dd,COUNTY_FIPS,STATE_NAME,CITY_STRING
0,1,479.0,Adamana,AZ,4,Apache,001,34.976692,-109.822338,4001,Arizona,Adamana
1,2,492.0,Adobe,AZ,4,Maricopa,013,33.689205,-112.122653,4013,Arizona,Adobe
2,3,511.0,Agua Fria,AZ,4,Maricopa,013,33.605595,-112.314604,4013,Arizona,Agua Fria
3,4,516.0,Aguila,AZ,4,Maricopa,013,33.942807,-113.174081,4013,Arizona,Aguila
4,5,538.0,Ajo,AZ,4,Pima,019,32.371725,-112.860710,4019,Arizona,Ajo


In [143]:
city_df['CITY_STRING'] = city_df['CITY_STRING'].str.replace("."," ")

In [144]:
city_df['CITY_STRING'] = city_df['CITY_STRING'].str.replace("St ","saint")

In [145]:
city_df['CITY_STRING'] = city_df['CITY_STRING'].str.replace("Township","")

In [146]:
city_df['CITY_STRING'] = city_df['CITY_STRING'].str.replace("Near ","")

In [147]:
city_df['CITY_STRING'] = city_df['CITY_STRING'].str.replace(",","")

In [ ]:
city_df['CITY_STRING'] = city_df['CITY_STRING'].str.replace("'","")

In [148]:
city_df['CITY_STRING'] = city_df['CITY_STRING'].str.replace("-","")

In [149]:
city_df['CITY_STRING'] = city_df['CITY_STRING'].str.replace("/","")

In [150]:
city_df['CITY_STRING'] = city_df['CITY_STRING'].str.replace("?","")

In [151]:
city_df['CITY_STRING'] = city_df['CITY_STRING'].str.replace(" ","")

In [152]:
city_df['CITY_STRING'] = city_df['CITY_STRING'].str.lower()
city_df.head()

,OBJECTID,FEATURE_ID,CITY_NAME,STUSPS,STATEFP,COUNTY_NAME,COUNTY_NUM,Lat_dd,Lon_dd,COUNTY_FIPS,STATE_NAME,CITY_STRING
0,1,479.0,Adamana,AZ,4,Apache,001,34.976692,-109.822338,4001,Arizona,adamana
1,2,492.0,Adobe,AZ,4,Maricopa,013,33.689205,-112.122653,4013,Arizona,adobe
2,3,511.0,Agua Fria,AZ,4,Maricopa,013,33.605595,-112.314604,4013,Arizona,aguafria
3,4,516.0,Aguila,AZ,4,Maricopa,013,33.942807,-113.174081,4013,Arizona,aguila
4,5,538.0,Ajo,AZ,4,Pima,019,32.371725,-112.860710,4019,Arizona,ajo


In [153]:
# make joint county_city column
city_df['COUNTYFP_CITY'] = city_df['COUNTY_FIPS'] + '_' + city_df['CITY_STRING']
city_df.head()

,OBJECTID,FEATURE_ID,CITY_NAME,STUSPS,STATEFP,COUNTY_NAME,COUNTY_NUM,Lat_dd,Lon_dd,COUNTY_FIPS,STATE_NAME,CITY_STRING,COUNTYFP_CITY
0,1,479.0,Adamana,AZ,4,Apache,001,34.976692,-109.822338,4001,Arizona,adamana,4001_adamana
1,2,492.0,Adobe,AZ,4,Maricopa,013,33.689205,-112.122653,4013,Arizona,adobe,4013_adobe
2,3,511.0,Agua Fria,AZ,4,Maricopa,013,33.605595,-112.314604,4013,Arizona,aguafria,4013_aguafria
3,4,516.0,Aguila,AZ,4,Maricopa,013,33.942807,-113.174081,4013,Arizona,aguila,4013_aguila
4,5,538.0,Ajo,AZ,4,Pima,019,32.371725,-112.860710,4019,Arizona,ajo,4019_ajo


In [154]:
# check length of dataframe
len(city_df)

200364

In [155]:
# if necessary export csv to check if duplicates [NOTE: yes - there were duplicates of the same city in the same county in the same state]
# city_df.to_csv('city_dups.csv', encoding='Windows-1252')

In [156]:
# get rid of duplicates
city_df = city_df.drop_duplicates(subset='COUNTYFP_CITY')
len(city_df)

199482

In [157]:
# make dictionary of COUNTYFP_CITY and city name string
city_dict = dict(zip(city_df.COUNTYFP_CITY, city_df.CITY_NAME))
# city_dict
# city_dict['4013_adobe']
len(city_dict)

199482

#### Map city_dict to missing_df to see if missing values

In [158]:
missing_df = missing_df.sort_values(by=['Case Number'])
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS
1979,MP1,10/1/1978,Allen,Anthony,16,Fort Smith,Sebastian,Arkansas,Male,White / Caucasian,2/13/2020,5,5_Sebastian,5131
14452,MP10,2/4/2000,Toliver,Charles,30,Clinton,Anderson,Tennessee,Male,White / Caucasian,8/5/2019,47,47_Anderson,47001
14969,MP100,8/16/1995,Butler,James,39,Kingsport,Sullivan,Tennessee,Male,White / Caucasian,7/29/2019,47,47_Sullivan,47163
8026,MP10000,8/20/2003,Hamilton,Carole,32,Owensboro,Daviess,Kentucky,Female,White / Caucasian,3/10/2020,21,21_Daviess,21059
11898,MP10002,12/13/2007,Ratynski,Edward,53,Fallsburg,Sullivan,New York,Male,White / Caucasian,1/6/2020,36,36_Sullivan,36105


In [159]:
# check dataframe
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS
1979,MP1,10/1/1978,Allen,Anthony,16,Fort Smith,Sebastian,Arkansas,Male,White / Caucasian,2/13/2020,5,5_Sebastian,5131
14452,MP10,2/4/2000,Toliver,Charles,30,Clinton,Anderson,Tennessee,Male,White / Caucasian,8/5/2019,47,47_Anderson,47001
14969,MP100,8/16/1995,Butler,James,39,Kingsport,Sullivan,Tennessee,Male,White / Caucasian,7/29/2019,47,47_Sullivan,47163
8026,MP10000,8/20/2003,Hamilton,Carole,32,Owensboro,Daviess,Kentucky,Female,White / Caucasian,3/10/2020,21,21_Daviess,21059
11898,MP10002,12/13/2007,Ratynski,Edward,53,Fallsburg,Sullivan,New York,Male,White / Caucasian,1/6/2020,36,36_Sullivan,36105


In [160]:
# check length of dataframe
len(missing_df)

17891

In [161]:
# make city names titled
missing_df['City_string'] = missing_df['City'].str.title()
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS,City_string
1979,MP1,10/1/1978,Allen,Anthony,16,Fort Smith,Sebastian,Arkansas,Male,White / Caucasian,2/13/2020,5,5_Sebastian,5131,Fort Smith
14452,MP10,2/4/2000,Toliver,Charles,30,Clinton,Anderson,Tennessee,Male,White / Caucasian,8/5/2019,47,47_Anderson,47001,Clinton
14969,MP100,8/16/1995,Butler,James,39,Kingsport,Sullivan,Tennessee,Male,White / Caucasian,7/29/2019,47,47_Sullivan,47163,Kingsport
8026,MP10000,8/20/2003,Hamilton,Carole,32,Owensboro,Daviess,Kentucky,Female,White / Caucasian,3/10/2020,21,21_Daviess,21059,Owensboro
11898,MP10002,12/13/2007,Ratynski,Edward,53,Fallsburg,Sullivan,New York,Male,White / Caucasian,1/6/2020,36,36_Sullivan,36105,Fallsburg


In [162]:
missing_df['City_string'] = missing_df['City_string'].str.replace("."," ")

In [163]:
missing_df['City_string'] = missing_df['City_string'].str.replace("Ft","fort")

In [164]:
missing_df['City_string'] = missing_df['City_string'].str.replace("St ","saint")

In [165]:
missing_df['City_string'] = missing_df['City_string'].str.replace("Ste ","saint")

In [166]:
missing_df['City_string'] = missing_df['City_string'].str.replace("Mt","mount")

In [167]:
missing_df['City_string'] = missing_df['City_string'].str.replace("^S ","south")

In [168]:
missing_df['City_string'] = missing_df['City_string'].str.replace("^E ","east")

In [169]:
missing_df['City_string'] = missing_df['City_string'].str.replace("^N ","north")

In [170]:
missing_df['City_string'] = missing_df['City_string'].str.replace("^W ","west")

In [171]:
missing_df['City_string'] = missing_df['City_string'].str.replace("Twp","")

In [172]:
missing_df['City_string'] = missing_df['City_string'].str.replace("Township","")

In [173]:
missing_df['City_string'] = missing_df['City_string'].str.replace("Near ","")

In [174]:
missing_df['City_string'] = missing_df['City_string'].str.replace("North Of ","")

In [175]:
missing_df['City_string'] = missing_df['City_string'].str.replace(" Dc","")

In [176]:
missing_df['City_string'] = missing_df['City_string'].str.replace(" District Of Columbia","")

In [177]:
missing_df['City_string'] = missing_df['City_string'].str.replace(",","")

In [178]:
missing_df['City_string'] = missing_df['City_string'].str.replace("'","")

In [179]:
missing_df['City_string'] = missing_df['City_string'].str.replace("-","")

In [180]:
missing_df['City_string'] = missing_df['City_string'].str.replace("/","")

In [181]:
missing_df['City_string'] = missing_df['City_string'].str.replace("?","")

In [182]:
missing_df['City_string'] = missing_df['City_string'].str.replace(" ","")

In [183]:
missing_df['City_string'] = missing_df['City_string'].str.replace("County","")

In [184]:
missing_df['City_string'] = missing_df['City_string'].str.lower()
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS,City_string
1979,MP1,10/1/1978,Allen,Anthony,16,Fort Smith,Sebastian,Arkansas,Male,White / Caucasian,2/13/2020,5,5_Sebastian,5131,fortsmith
14452,MP10,2/4/2000,Toliver,Charles,30,Clinton,Anderson,Tennessee,Male,White / Caucasian,8/5/2019,47,47_Anderson,47001,clinton
14969,MP100,8/16/1995,Butler,James,39,Kingsport,Sullivan,Tennessee,Male,White / Caucasian,7/29/2019,47,47_Sullivan,47163,kingsport
8026,MP10000,8/20/2003,Hamilton,Carole,32,Owensboro,Daviess,Kentucky,Female,White / Caucasian,3/10/2020,21,21_Daviess,21059,owensboro
11898,MP10002,12/13/2007,Ratynski,Edward,53,Fallsburg,Sullivan,New York,Male,White / Caucasian,1/6/2020,36,36_Sullivan,36105,fallsburg


In [185]:
missing_df['City_string'] = missing_df['City_string'].str.replace('unknown','', regex=False)

In [186]:
missing_df['City_string'] = missing_df['City_string'].str.replace('undetermined','', regex=False)

In [187]:
missing_df['City_string'] = missing_df['City_string'].str.replace('.*\d.*', '', regex=True)

In [188]:
missing_df['City_string'] = missing_df['City_string'].replace(r'^\s*$', np.NaN, regex=True)
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS,City_string
1979,MP1,10/1/1978,Allen,Anthony,16,Fort Smith,Sebastian,Arkansas,Male,White / Caucasian,2/13/2020,5,5_Sebastian,5131,fortsmith
14452,MP10,2/4/2000,Toliver,Charles,30,Clinton,Anderson,Tennessee,Male,White / Caucasian,8/5/2019,47,47_Anderson,47001,clinton
14969,MP100,8/16/1995,Butler,James,39,Kingsport,Sullivan,Tennessee,Male,White / Caucasian,7/29/2019,47,47_Sullivan,47163,kingsport
8026,MP10000,8/20/2003,Hamilton,Carole,32,Owensboro,Daviess,Kentucky,Female,White / Caucasian,3/10/2020,21,21_Daviess,21059,owensboro
11898,MP10002,12/13/2007,Ratynski,Edward,53,Fallsburg,Sullivan,New York,Male,White / Caucasian,1/6/2020,36,36_Sullivan,36105,fallsburg


In [189]:
# check null values [NOTE: ]
missing_citystring_null_df = missing_df.loc[missing_df['City_string'].isnull()]
# missing_citystring_null_df.shape
missing_citystring_null_df

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS,City_string
7971,MP12696,6/12/1981,Mckibben,Phillip,35,unknown,Adair,Kentucky,Male,White / Caucasian,4/2/2018,21,21_Adair,21001,NaN
9991,MP13006,5/28/1977,Bullock,Janis,26,NaN,Pearl River,Mississippi,Female,White / Caucasian,2/24/2020,28,28_Pearl River,28109,NaN
1589,MP15642,1/1/2006,Betancourt guzman,Lorenzo,28,Unknown,Pima,Arizona,Male,Hispanic / Latino,10/5/2017,4,4_Pima,4019,NaN
1591,MP15676,1/1/2001,Rodriguez,Juan,31,Unknown,Pima,Arizona,Male,Hispanic / Latino,12/19/2017,4,4_Pima,4019,NaN
1590,MP15677,1/1/2001,Reynosa,Martin,31,unknown,Pima,Arizona,Male,Hispanic / Latino,12/19/2017,4,4_Pima,4019,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15023,MP9088,3/1/1983,Douglass,Ronald,39,Unknown,Austin,Texas,Male,White / Caucasian,3/4/2020,48,48_Austin,48015,NaN
16433,MP9206,11/9/2009,Jones,Creola,34,NaN,St. Croix,United States Virgin Islands,Female,Black / African American,9/17/2012,78,78_St. Croix,78010,NaN
3644,MP9467,6/17/2004,Collier,Terry,55,Unknown,San Bernardino,California,Female,White / Caucasian,4/29/2019,6,6_San Bernardino,6071,NaN
5169,MP9544,8/17/1999,Sayre,Tom,37,Unknown,Alachua,Florida,Male,White / Caucasian,3/12/2020,12,12_Alachua,12001,NaN


In [190]:
# Add column with compound field key
missing_df['County_city'] = missing_df['County_FIPS'].astype(str) + "_" + missing_df['City_string']
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS,City_string,County_city
1979,MP1,10/1/1978,Allen,Anthony,16,Fort Smith,Sebastian,Arkansas,Male,White / Caucasian,2/13/2020,5,5_Sebastian,5131,fortsmith,5131_fortsmith
14452,MP10,2/4/2000,Toliver,Charles,30,Clinton,Anderson,Tennessee,Male,White / Caucasian,8/5/2019,47,47_Anderson,47001,clinton,47001_clinton
14969,MP100,8/16/1995,Butler,James,39,Kingsport,Sullivan,Tennessee,Male,White / Caucasian,7/29/2019,47,47_Sullivan,47163,kingsport,47163_kingsport
8026,MP10000,8/20/2003,Hamilton,Carole,32,Owensboro,Daviess,Kentucky,Female,White / Caucasian,3/10/2020,21,21_Daviess,21059,owensboro,21059_owensboro
11898,MP10002,12/13/2007,Ratynski,Edward,53,Fallsburg,Sullivan,New York,Male,White / Caucasian,1/6/2020,36,36_Sullivan,36105,fallsburg,36105_fallsburg


In [191]:
# Add new column to dataframe, using the County_city name field as a key in the city_dict, to pull the GNIS city name for each row
missing_df['GNIS_city'] = missing_df['County_city'].map(city_dict)
missing_df.head()

,Case Number,DLC,Last Name,First Name,Missing Age,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS,City_string,County_city,GNIS_city
1979,MP1,10/1/1978,Allen,Anthony,16,Fort Smith,Sebastian,Arkansas,Male,White / Caucasian,2/13/2020,5,5_Sebastian,5131,fortsmith,5131_fortsmith,Fort Smith
14452,MP10,2/4/2000,Toliver,Charles,30,Clinton,Anderson,Tennessee,Male,White / Caucasian,8/5/2019,47,47_Anderson,47001,clinton,47001_clinton,Clinton
14969,MP100,8/16/1995,Butler,James,39,Kingsport,Sullivan,Tennessee,Male,White / Caucasian,7/29/2019,47,47_Sullivan,47163,kingsport,47163_kingsport,Kingsport
8026,MP10000,8/20/2003,Hamilton,Carole,32,Owensboro,Daviess,Kentucky,Female,White / Caucasian,3/10/2020,21,21_Daviess,21059,owensboro,21059_owensboro,Owensboro
11898,MP10002,12/13/2007,Ratynski,Edward,53,Fallsburg,Sullivan,New York,Male,White / Caucasian,1/6/2020,36,36_Sullivan,36105,fallsburg,36105_fallsburg,Fallsburg


In [192]:
# Check unique values in new dataframe field
missing_df['GNIS_city'].unique()

array(['Fort Smith', 'Clinton', 'Kingsport', ..., 'Owego', 'Uniontown',
       'Virgie'], dtype=object)

In [193]:
# check null values [NOTE: ]
missing_city_null_df = missing_df.loc[missing_df['GNIS_city'].isnull()]
missing_city_null_df.shape
# missing_city_null_df

(1720, 17)

In [195]:
# As needed, export nulls to address [NOTE:]
missing_city_null_df.to_csv('missing_city_nulls.csv', encoding='Windows-1252')

#### Map city_dict to unclaimed_df to see if missing values

In [197]:
unclaimed_df = unclaimed_df.sort_values(by=['DBF'])
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS,State_County,County_FIPS,City_string
7042,UCP3482,1/1/2004,Green,Ellen,Female,White / Caucasian,Orange,Orange,California,4/18/2017,6,6_Orange,6059,Orange
6941,UCP3523,1/1/2005,Baldwin,Bette,Female,White / Caucasian,San Juan Capistrano,Orange,California,4/20/2017,6,6_Orange,6059,San Juan Capistrano
6815,UCP853,1/1/2006,Eckberg,Robert,Male,White / Caucasian,NaN,Spokane,Washington,4/14/2020,53,53_Spokane,53063,NaN
6654,UCP63328,1/1/2007,Bishop,Robert,Unknown,Uncertain,NaN,San Bernardino,California,12/19/2019,6,6_San Bernardino,6071,NaN
6655,UCP3793,1/1/2007,Lopez,Agnes,Female,White / Caucasian,Fullerton,Orange,California,5/4/2017,6,6_Orange,6059,Fullerton


In [198]:
# check dataframe
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS,State_County,County_FIPS,City_string
7042,UCP3482,1/1/2004,Green,Ellen,Female,White / Caucasian,Orange,Orange,California,4/18/2017,6,6_Orange,6059,Orange
6941,UCP3523,1/1/2005,Baldwin,Bette,Female,White / Caucasian,San Juan Capistrano,Orange,California,4/20/2017,6,6_Orange,6059,San Juan Capistrano
6815,UCP853,1/1/2006,Eckberg,Robert,Male,White / Caucasian,NaN,Spokane,Washington,4/14/2020,53,53_Spokane,53063,NaN
6654,UCP63328,1/1/2007,Bishop,Robert,Unknown,Uncertain,NaN,San Bernardino,California,12/19/2019,6,6_San Bernardino,6071,NaN
6655,UCP3793,1/1/2007,Lopez,Agnes,Female,White / Caucasian,Fullerton,Orange,California,5/4/2017,6,6_Orange,6059,Fullerton


In [199]:
# check length of dataframe
len(unclaimed_df)

8335

In [200]:
# make city names titled
unclaimed_df['City_string'] = unclaimed_df['City'].str.title()
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS,State_County,County_FIPS,City_string
7042,UCP3482,1/1/2004,Green,Ellen,Female,White / Caucasian,Orange,Orange,California,4/18/2017,6,6_Orange,6059,Orange
6941,UCP3523,1/1/2005,Baldwin,Bette,Female,White / Caucasian,San Juan Capistrano,Orange,California,4/20/2017,6,6_Orange,6059,San Juan Capistrano
6815,UCP853,1/1/2006,Eckberg,Robert,Male,White / Caucasian,NaN,Spokane,Washington,4/14/2020,53,53_Spokane,53063,NaN
6654,UCP63328,1/1/2007,Bishop,Robert,Unknown,Uncertain,NaN,San Bernardino,California,12/19/2019,6,6_San Bernardino,6071,NaN
6655,UCP3793,1/1/2007,Lopez,Agnes,Female,White / Caucasian,Fullerton,Orange,California,5/4/2017,6,6_Orange,6059,Fullerton


In [201]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].str.replace("."," ")

In [202]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].str.replace("Ft","fort")

In [203]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].str.replace("St ","saint")

In [204]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].str.replace("Ste ","saint")

In [205]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].str.replace("Mt","mount")

In [206]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].str.replace("^S ","south")

In [207]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].str.replace("^E ","east")

In [208]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].str.replace("^N ","north")

In [209]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].str.replace("^W ","west")

In [210]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].str.replace("Twp","")

In [211]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].str.replace("Township","")

In [212]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].str.replace("Near ","")

In [213]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].str.replace("North Of ","")

In [214]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].str.replace(" Dc","")

In [215]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].str.replace(" District Of Columbia","")

In [216]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].str.replace(",","")

In [217]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].str.replace("'","")

In [218]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].str.replace("-","")

In [219]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].str.replace("/","")

In [220]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].str.replace("?","")

In [221]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].str.replace(" ","")

In [222]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].str.replace("County","")

In [223]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].str.lower()
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS,State_County,County_FIPS,City_string
7042,UCP3482,1/1/2004,Green,Ellen,Female,White / Caucasian,Orange,Orange,California,4/18/2017,6,6_Orange,6059,orange
6941,UCP3523,1/1/2005,Baldwin,Bette,Female,White / Caucasian,San Juan Capistrano,Orange,California,4/20/2017,6,6_Orange,6059,sanjuancapistrano
6815,UCP853,1/1/2006,Eckberg,Robert,Male,White / Caucasian,NaN,Spokane,Washington,4/14/2020,53,53_Spokane,53063,NaN
6654,UCP63328,1/1/2007,Bishop,Robert,Unknown,Uncertain,NaN,San Bernardino,California,12/19/2019,6,6_San Bernardino,6071,NaN
6655,UCP3793,1/1/2007,Lopez,Agnes,Female,White / Caucasian,Fullerton,Orange,California,5/4/2017,6,6_Orange,6059,fullerton


In [224]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].str.replace('unknown','', regex=False)

In [225]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].str.replace('undetermined','', regex=False)

In [226]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].str.replace('.*\d.*', '', regex=True)

In [227]:
unclaimed_df['City_string'] = unclaimed_df['City_string'].replace(r'^\s*$', np.NaN, regex=True)
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS,State_County,County_FIPS,City_string
7042,UCP3482,1/1/2004,Green,Ellen,Female,White / Caucasian,Orange,Orange,California,4/18/2017,6,6_Orange,6059,orange
6941,UCP3523,1/1/2005,Baldwin,Bette,Female,White / Caucasian,San Juan Capistrano,Orange,California,4/20/2017,6,6_Orange,6059,sanjuancapistrano
6815,UCP853,1/1/2006,Eckberg,Robert,Male,White / Caucasian,NaN,Spokane,Washington,4/14/2020,53,53_Spokane,53063,NaN
6654,UCP63328,1/1/2007,Bishop,Robert,Unknown,Uncertain,NaN,San Bernardino,California,12/19/2019,6,6_San Bernardino,6071,NaN
6655,UCP3793,1/1/2007,Lopez,Agnes,Female,White / Caucasian,Fullerton,Orange,California,5/4/2017,6,6_Orange,6059,fullerton


In [229]:
# check null values [NOTE: ]
unclaimed_citystring_null_df = unclaimed_df.loc[unclaimed_df['City_string'].isnull()]
# unclaimed_citystring_null_df.shape
unclaimed_citystring_null_df

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS,State_County,County_FIPS,City_string
6815,UCP853,1/1/2006,Eckberg,Robert,Male,White / Caucasian,NaN,Spokane,Washington,4/14/2020,53,53_Spokane,53063,NaN
6654,UCP63328,1/1/2007,Bishop,Robert,Unknown,Uncertain,NaN,San Bernardino,California,12/19/2019,6,6_San Bernardino,6071,NaN
6198,UCP63351,1/1/2009,Johnston,James,Unknown,Uncertain,NaN,San Bernardino,California,12/19/2019,6,6_San Bernardino,6071,NaN
6200,UCP63533,1/1/2009,Leyva,Melvis,Unknown,Uncertain,NaN,San Bernardino,California,12/19/2019,6,6_San Bernardino,6071,NaN
5935,UCP63397,1/1/2010,Mccormack,Robert,Unknown,Uncertain,NaN,San Bernardino,California,12/19/2019,6,6_San Bernardino,6071,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8329,UCP1040,NaN,Snell,Gerald,Male,Uncertain,NaN,Wayne,Michigan,3/19/2020,26,26_Wayne,26163,NaN
8331,UCP341,NaN,Brotherton,Quilen,Male,Uncertain,NaN,Spokane,Washington,4/14/2020,53,53_Spokane,53063,NaN
8332,UCP326,NaN,Bell,Phillip,Male,Uncertain,NaN,Spokane,Washington,4/14/2020,53,53_Spokane,53063,NaN
8333,UCP345,NaN,Buonantony,Elena,Female,Uncertain,NaN,Spokane,Washington,4/14/2020,53,53_Spokane,53063,NaN


In [230]:
# Add column with compound field key
unclaimed_df['County_city'] = unclaimed_df['County_FIPS'].astype(str) + "_" + unclaimed_df['City_string']
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS,State_County,County_FIPS,City_string,County_city
7042,UCP3482,1/1/2004,Green,Ellen,Female,White / Caucasian,Orange,Orange,California,4/18/2017,6,6_Orange,6059,orange,6059_orange
6941,UCP3523,1/1/2005,Baldwin,Bette,Female,White / Caucasian,San Juan Capistrano,Orange,California,4/20/2017,6,6_Orange,6059,sanjuancapistrano,6059_sanjuancapistrano
6815,UCP853,1/1/2006,Eckberg,Robert,Male,White / Caucasian,NaN,Spokane,Washington,4/14/2020,53,53_Spokane,53063,NaN,NaN
6654,UCP63328,1/1/2007,Bishop,Robert,Unknown,Uncertain,NaN,San Bernardino,California,12/19/2019,6,6_San Bernardino,6071,NaN,NaN
6655,UCP3793,1/1/2007,Lopez,Agnes,Female,White / Caucasian,Fullerton,Orange,California,5/4/2017,6,6_Orange,6059,fullerton,6059_fullerton


In [231]:
# Add new column to dataframe, using the County_city name field as a key in the city_dict, to pull the GNIS city name for each row
unclaimed_df['GNIS_city'] = unclaimed_df['County_city'].map(city_dict)
unclaimed_df.head()

,Case Number,DBF,Last Name,First Name,Sex,Race / Ethnicity,City,County,State,Date Modified,State_FIPS,State_County,County_FIPS,City_string,County_city,GNIS_city
7042,UCP3482,1/1/2004,Green,Ellen,Female,White / Caucasian,Orange,Orange,California,4/18/2017,6,6_Orange,6059,orange,6059_orange,Orange
6941,UCP3523,1/1/2005,Baldwin,Bette,Female,White / Caucasian,San Juan Capistrano,Orange,California,4/20/2017,6,6_Orange,6059,sanjuancapistrano,6059_sanjuancapistrano,San Juan Capistrano
6815,UCP853,1/1/2006,Eckberg,Robert,Male,White / Caucasian,NaN,Spokane,Washington,4/14/2020,53,53_Spokane,53063,NaN,NaN,NaN
6654,UCP63328,1/1/2007,Bishop,Robert,Unknown,Uncertain,NaN,San Bernardino,California,12/19/2019,6,6_San Bernardino,6071,NaN,NaN,NaN
6655,UCP3793,1/1/2007,Lopez,Agnes,Female,White / Caucasian,Fullerton,Orange,California,5/4/2017,6,6_Orange,6059,fullerton,6059_fullerton,Fullerton


In [232]:
# Check unique values in new dataframe field
unclaimed_df['GNIS_city'].unique()

array(['Orange', 'San Juan Capistrano', nan, 'Fullerton', 'Anaheim',
       'Portland', 'Santa Ana', 'Yakima', 'Boise', 'Oklahoma City',
       'New York', 'Bronx', 'Atlanta', 'Garberville', 'Waipahu',
       'Gig Harbor', 'Knoxville', 'Reno', 'Westminster', 'Tacoma',
       'Norfolk', 'Brooklyn', 'Queens', 'Springfield', 'Chattanooga',
       'Honolulu', 'Framingham', 'Chula Vista', 'West Haven',
       'Richmond Hill', 'East Elmhurst', 'Far Rockaway', 'Anderson',
       'Cambridge', 'Everett', 'Houston', 'El Centro', 'Staten Island',
       'University Place', 'Merced', 'Spokane', 'Stanton', 'Los Angeles',
       'Worcester', 'Natchitoches', 'El Paso', 'Detroit', 'Flushing',
       'Northampton', 'Oakland', 'Garden City', 'Philadelphia',
       'San Diego', 'Fountain Valley', 'Wilmington', 'Boston', 'McAllen',
       'Hoquiam', 'Steilacoom', 'Marathon', 'Millington', 'Seattle',
       'Reading', 'Mansfield', 'Buena Park', 'El Cajon', 'Lakewood',
       'Huntington Beach', 'Muskogee',

In [233]:
# check null values [NOTE: ]
unclaimed_city_null_df = unclaimed_df.loc[unclaimed_df['GNIS_city'].isnull()]
unclaimed_city_null_df.shape
# unclaimed_city_null_df

(3078, 16)

In [234]:
# As needed, export nulls to address [NOTE:]
unclaimed_city_null_df.to_csv('unclaimed_city_nulls.csv', encoding='Windows-1252')

#### Map city_dict to unidentified_df to see if missing values

In [235]:
# check dataframe
unidentified_df.head()

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS
0,UP51903,3/30/1986,0.0,120.0,Marbury,Autauga,Alabama,Female,White / Caucasian,1/28/2020,1,1_Autauga,1001
1,UP5000,9/29/2007,40.0,60.0,Prattville,Autauga,Alabama,Female,Black / African American,8/12/2019,1,1_Autauga,1001
2,UP6519,1/6/2006,45.0,59.0,Elberta,Baldwin,Alabama,Male,White / Caucasian,10/24/2018,1,1_Baldwin,1003
3,UP55409,3/26/2000,0.0,120.0,NaN,Baldwin,Alabama,Male,"White / Caucasian, Asian, American Indian / Al...",2/13/2020,1,1_Baldwin,1003
4,UP13483,3/27/1961,14.0,17.0,NaN,Bibb,Alabama,Male,White / Caucasian,3/25/2019,1,1_Bibb,1007


In [236]:
# check length of dataframe
len(unidentified_df)

13260

In [237]:
# make city names titled
unidentified_df['City_string'] = unidentified_df['City'].str.title()
unidentified_df.head()

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS,City_string
0,UP51903,3/30/1986,0.0,120.0,Marbury,Autauga,Alabama,Female,White / Caucasian,1/28/2020,1,1_Autauga,1001,Marbury
1,UP5000,9/29/2007,40.0,60.0,Prattville,Autauga,Alabama,Female,Black / African American,8/12/2019,1,1_Autauga,1001,Prattville
2,UP6519,1/6/2006,45.0,59.0,Elberta,Baldwin,Alabama,Male,White / Caucasian,10/24/2018,1,1_Baldwin,1003,Elberta
3,UP55409,3/26/2000,0.0,120.0,NaN,Baldwin,Alabama,Male,"White / Caucasian, Asian, American Indian / Al...",2/13/2020,1,1_Baldwin,1003,NaN
4,UP13483,3/27/1961,14.0,17.0,NaN,Bibb,Alabama,Male,White / Caucasian,3/25/2019,1,1_Bibb,1007,NaN


In [238]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace("."," ")

In [239]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace("Ft","fort")

In [240]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace("St ","saint")

In [241]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace("Ste ","saint")

In [242]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace("Mt","mount")

In [243]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace("^S ","south")

In [244]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace("^E ","east")

In [245]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace("^N ","north")

In [246]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace("^W ","west")

In [247]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace("Twp","")

In [248]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace("Township","")

In [249]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace("Near ","")

In [250]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace("Vicinity Of ","")

In [251]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace("North Of ","")

In [252]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace("Northeast Of ","")

In [253]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace("East Of ","")

In [254]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace("Southeast Of ","")

In [255]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace("South Of ","")

In [256]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace("Southwest Of ","")

In [257]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace("West Of ","")

In [258]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace("Northwest Of ","")

In [259]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace(" Vicinity","")

In [260]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace(" Dc","")

In [261]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace(" District Of Columbia","")

In [262]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace(",","")

In [263]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace("'","")

In [264]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace("-","")

In [265]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace("/","")

In [266]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace("?","")

In [267]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace(" ","")

In [268]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace("County","")

In [269]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.lower()
unidentified_df.head()

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS,City_string
0,UP51903,3/30/1986,0.0,120.0,Marbury,Autauga,Alabama,Female,White / Caucasian,1/28/2020,1,1_Autauga,1001,marbury
1,UP5000,9/29/2007,40.0,60.0,Prattville,Autauga,Alabama,Female,Black / African American,8/12/2019,1,1_Autauga,1001,prattville
2,UP6519,1/6/2006,45.0,59.0,Elberta,Baldwin,Alabama,Male,White / Caucasian,10/24/2018,1,1_Baldwin,1003,elberta
3,UP55409,3/26/2000,0.0,120.0,NaN,Baldwin,Alabama,Male,"White / Caucasian, Asian, American Indian / Al...",2/13/2020,1,1_Baldwin,1003,NaN
4,UP13483,3/27/1961,14.0,17.0,NaN,Bibb,Alabama,Male,White / Caucasian,3/25/2019,1,1_Bibb,1007,NaN


In [270]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace('unknown','', regex=False)

In [271]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace('undetermined','', regex=False)

In [272]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace('unincorporated','', regex=False)

In [273]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace('rural','', regex=False)

In [274]:
unidentified_df['City_string'] = unidentified_df['City_string'].str.replace('.*\d.*', '', regex=True)

In [275]:
unidentified_df['City_string'] = unidentified_df['City_string'].replace(r'^\s*$', np.NaN, regex=True)
unidentified_df.head()

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS,City_string
0,UP51903,3/30/1986,0.0,120.0,Marbury,Autauga,Alabama,Female,White / Caucasian,1/28/2020,1,1_Autauga,1001,marbury
1,UP5000,9/29/2007,40.0,60.0,Prattville,Autauga,Alabama,Female,Black / African American,8/12/2019,1,1_Autauga,1001,prattville
2,UP6519,1/6/2006,45.0,59.0,Elberta,Baldwin,Alabama,Male,White / Caucasian,10/24/2018,1,1_Baldwin,1003,elberta
3,UP55409,3/26/2000,0.0,120.0,NaN,Baldwin,Alabama,Male,"White / Caucasian, Asian, American Indian / Al...",2/13/2020,1,1_Baldwin,1003,NaN
4,UP13483,3/27/1961,14.0,17.0,NaN,Bibb,Alabama,Male,White / Caucasian,3/25/2019,1,1_Bibb,1007,NaN


In [276]:
# check null values [NOTE: ]
unidentified_citystring_null_df = unidentified_df.loc[unidentified_df['City_string'].isnull()]
# unidentified_citystring_null_df.shape
unidentified_citystring_null_df

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS,City_string
3,UP55409,3/26/2000,0.0,120.0,NaN,Baldwin,Alabama,Male,"White / Caucasian, Asian, American Indian / Al...",2/13/2020,1,1_Baldwin,1003,NaN
4,UP13483,3/27/1961,14.0,17.0,NaN,Bibb,Alabama,Male,White / Caucasian,3/25/2019,1,1_Bibb,1007,NaN
5,UP512,10/31/2005,33.0,43.0,NaN,Blount,Alabama,Male,White / Caucasian,5/28/2018,1,1_Blount,1009,NaN
32,UP13161,11/3/1982,29.0,32.0,NaN,Lee,Alabama,Female,Black / African American,7/10/2015,1,1_Lee,1081,NaN
38,UP888,4/15/1997,19.0,34.0,NaN,Marshall,Alabama,Male,White / Caucasian,6/8/2018,1,1_Marshall,1095,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13217,UP4762,8/27/1993,39.0,60.0,NaN,Kenosha,Wisconsin,Male,Uncertain,12/7/2018,55,55_Kenosha,55059,NaN
13253,UP13898,4/29/2015,40.0,65.0,NaN,Winnebago,Wisconsin,Male,White / Caucasian,1/29/2020,55,55_Winnebago,55139,NaN
13257,UP9970,4/13/1992,16.0,21.0,NaN,Sheridan,Wyoming,Female,White / Caucasian,3/20/2020,56,56_Sheridan,56033,NaN
13258,UP9973,8/27/1982,23.0,28.0,NaN,Sweetwater,Wyoming,Male,White / Caucasian,3/20/2020,56,56_Sweetwater,56037,NaN


In [277]:
# Add column with compound field key
unidentified_df['County_city'] = unidentified_df['County_FIPS'].astype(str) + "_" + unidentified_df['City_string']
unidentified_df.head()

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS,City_string,County_city
0,UP51903,3/30/1986,0.0,120.0,Marbury,Autauga,Alabama,Female,White / Caucasian,1/28/2020,1,1_Autauga,1001,marbury,1001_marbury
1,UP5000,9/29/2007,40.0,60.0,Prattville,Autauga,Alabama,Female,Black / African American,8/12/2019,1,1_Autauga,1001,prattville,1001_prattville
2,UP6519,1/6/2006,45.0,59.0,Elberta,Baldwin,Alabama,Male,White / Caucasian,10/24/2018,1,1_Baldwin,1003,elberta,1003_elberta
3,UP55409,3/26/2000,0.0,120.0,NaN,Baldwin,Alabama,Male,"White / Caucasian, Asian, American Indian / Al...",2/13/2020,1,1_Baldwin,1003,NaN,NaN
4,UP13483,3/27/1961,14.0,17.0,NaN,Bibb,Alabama,Male,White / Caucasian,3/25/2019,1,1_Bibb,1007,NaN,NaN


In [278]:
# Add new column to dataframe, using the County_city name field as a key in the city_dict, to pull the GNIS city name for each row
unidentified_df['GNIS_city'] = unidentified_df['County_city'].map(city_dict)
unidentified_df.head()

,Case Number,DBF,Age From,Age To,City,County,State,Sex,Race / Ethnicity,Date Modified,State_FIPS,State_County,County_FIPS,City_string,County_city,GNIS_city
0,UP51903,3/30/1986,0.0,120.0,Marbury,Autauga,Alabama,Female,White / Caucasian,1/28/2020,1,1_Autauga,1001,marbury,1001_marbury,Marbury
1,UP5000,9/29/2007,40.0,60.0,Prattville,Autauga,Alabama,Female,Black / African American,8/12/2019,1,1_Autauga,1001,prattville,1001_prattville,Prattville
2,UP6519,1/6/2006,45.0,59.0,Elberta,Baldwin,Alabama,Male,White / Caucasian,10/24/2018,1,1_Baldwin,1003,elberta,1003_elberta,Elberta
3,UP55409,3/26/2000,0.0,120.0,NaN,Baldwin,Alabama,Male,"White / Caucasian, Asian, American Indian / Al...",2/13/2020,1,1_Baldwin,1003,NaN,NaN,NaN
4,UP13483,3/27/1961,14.0,17.0,NaN,Bibb,Alabama,Male,White / Caucasian,3/25/2019,1,1_Bibb,1007,NaN,NaN,NaN


In [279]:
# Check unique values in new dataframe field
unidentified_df['GNIS_city'].unique()

array(['Marbury', 'Prattville', 'Elberta', ..., 'Foxpark', 'Wright',
       'Cheyenne'], dtype=object)

In [280]:
# check null values [NOTE: ]
unidentified_city_null_df = unidentified_df.loc[unidentified_df['GNIS_city'].isnull()]
unidentified_city_null_df.shape
# unidentified_city_null_df

(3655, 16)

In [281]:
# As needed, export nulls to address [NOTE:]
unidentified_city_null_df.to_csv('unidentified_city_nulls.csv', encoding='Windows-1252')

# PART 5 - construct state-, county-, and city-level GeoJSON files

### Construct State-level GeoJSON with correct structure

Goal format:
{
"type": "Feature",
    "name": "Wisconsin",
    "properties": {
        "missing": [ ],
        "unclaimed": [ ],
        "unidentified": [ ],
        "filtered": [ ]
    }
    "geometry": {
          "type": "Point",
          "coordinates": [
            -117.79750667,
            36.03755926
          ]
}
* each array will be a list of dictionaries. Each dictionary = one case. Keys = headers

In [ ]:
# get headers
missing_header = list(missing_df.columns.values)
print("missing header:", missing_header)
unclaimed_header = list(unclaimed_df.columns.values)
print("unclaimed header:", unclaimed_header)
unidentified_header = list(unidentified_df.columns.values)
print("unidentified header:", unidentified_header)

In [ ]:
# check df
unclaimed_df.head()

In [ ]:
# check data types
type(unclaimed_df['State_FIPS'][0])

In [ ]:
# check centroids
state_centroids_v2_df.head()
len(state_centroids_v2_df)

In [ ]:
# sort each database by state FIPS
# sort state centroids by state FIPS
state_centroids_v2_df = state_centroids_v2_df.sort_values(by=['STATEFP'])
state_centroids_v2_df.head()

In [ ]:
# sort each database by state FIPS
# sort missing by state FIPS
missing_df = missing_df.sort_values(by=['State_FIPS'])
missing_df.head()

In [ ]:
# sort unclaimed by state FIPS
unclaimed_df = unclaimed_df.sort_values(by=['State_FIPS'])
unclaimed_df.head()

In [ ]:
# sort unidentified by state FIPS
unidentified_df = unidentified_df.sort_values(by=['State_FIPS'])
unidentified_df.head()

In [ ]:
## Create state array
state_array = []
## for each state in state_centroids...
i = 0
while i < len(state_centroids_v2_df):
    state_dict = {}
    state_dict["type"] = "Feature"
    state_dict["name"] = state_centroids_v2_df["NAME"][i]
    state_dict["name_abbr"] = state_centroids_v2_df["STUSPS"][i]
    state_dict["state_FIPS"] = str(state_centroids_v2_df["STATEFP"][i])
    state_dict["properties"] = {}
    missing_array = []
    j = 0
    while j < len(missing_df):
        # check if state_fips matches
        if missing_df['State_FIPS'][j] == state_centroids_v2_df["STATEFP"][i]:
            missing_dict = {}
            for item in missing_header:
                missing_dict[item] = str(missing_df[item][j])
            # append dictionary to missing array
            missing_array.append(missing_dict)
        # increment j
        j += 1
    state_dict["properties"]['missing'] = missing_array     
    
    unclaimed_array = []
    k = 0
    while k < len(unclaimed_df):
        # check if state_fips matches
        if unclaimed_df['State_FIPS'][k] == state_centroids_v2_df["STATEFP"][i]:
            unclaimed_dict = {}
            for item in unclaimed_header:
                unclaimed_dict[item] = str(unclaimed_df[item][k])
            # append dictionary to unclaimed array
            unclaimed_array.append(unclaimed_dict)
        # increment k
        k += 1
    state_dict["properties"]['unclaimed'] = unclaimed_array
    
    unidentified_array = []
    l = 0
    while l < len(unidentified_df):
        # check if state_fips matches
        if unidentified_df['State_FIPS'][l] == state_centroids_v2_df["STATEFP"][i]:
            unidentified_dict = {}
            for item in unidentified_header:
                unidentified_dict[item] = str(unidentified_df[item][l])
            # append dictionary to unclaimed array
            unidentified_array.append(unidentified_dict)
        # increment l
        l += 1
    state_dict["properties"]['unidentified'] = unidentified_array
    
    state_dict["properties"]['filtered'] = []
    ## set geometry
    state_dict["geometry"] = {}
    state_dict["geometry"]["type"] = "Point"
    state_dict["geometry"]["coordinates"] = [state_centroids_v2_df['Lon_dd'][i], state_centroids_v2_df['Lat_dd'][i]]
    ## append state dictionary to array
    state_array.append(state_dict)
    # increment interator
    i += 1
# state_array

In [ ]:
# check item in array
len(state_array)
# state_array[0]['properties']['unclaimed']

In [ ]:
# Create FeatureCollection
state_feature_collection = {}
state_feature_collection["type"] = "FeatureCollection"
state_feature_collection["features"] = state_array
# state_feature_collection

In [ ]:
# Convert FeatureCollection to JSON format
state_geojson = geojson.dumps(state_feature_collection)
# check type to make sure conversion was sucessful
print(type(state_geojson))

In [ ]:
# Save JSON-formatted FeatureCollection as JSON file
# Save as new json file
with open('JSON/state_geojson.json', 'w', encoding='utf-8') as json_file:
    json_file.write(state_geojson)

### Construct COUNTY-LEVEL GeoJSON with correct structure

Goal format:
{
"type": "Feature",
    "name": "Dane",
    "state": "Wisconsin",
    "state_FIPS": ,
    "county_FIPS": ,
    "properties": {
        "missing": [ ],
        "unclaimed": [ ],
        "unidentified": [ ],
        "filtered": [ ]
    }
    "geometry": {
          "type": "Point",
          "coordinates": [
            -117.79750667,
            36.03755926
          ]
}
* each array will be a list of dictionaries. Each dictionary = one case. Keys = headers

In [ ]:
# get headers
missing_header = list(missing_df.columns.values)
print("missing header:", missing_header)
unclaimed_header = list(unclaimed_df.columns.values)
print("unclaimed header:", unclaimed_header)
unidentified_header = list(unidentified_df.columns.values)
print("unidentified header:", unidentified_header)

In [ ]:
# check df
unclaimed_df.head()

In [ ]:
# check data types
type(unclaimed_df['State_FIPS'][0])

In [ ]:
# check centroids
county_centroids_v2_df.head()
len(county_centroids_v2_df)

In [ ]:
# sort each database by county FIPS
# sort county centroids by county FIPS
county_centroids_v2_df = county_centroids_v2_df.sort_values(by=['GEOID'])
county_centroids_v2_df.head()

In [ ]:
# sort each database by county FIPS
# sort missing by county FIPS
missing_df = missing_df.sort_values(by=['County_FIPS'])
missing_df.head()

In [ ]:
# sort unclaimed by county FIPS
unclaimed_df = unclaimed_df.sort_values(by=['County_FIPS'])
unclaimed_df.head()

In [ ]:
# sort unidentified by county FIPS
unidentified_df = unidentified_df.sort_values(by=['County_FIPS'])
unidentified_df.head()

In [ ]:
## Create county array 
county_array = []
## for each county in county_centroids_v2...
i = 0
while i < len(county_centroids_v2_df):
    county_dict = {}
    county_dict["type"] = "Feature"
    county_dict["name"] = str(county_centroids_v2_df["NAME"][i])
    county_dict["county_FIPS"] = str(county_centroids_v2_df["GEOID"][i])
    county_dict["state_name"] = str(county_centroids_v2_df["STATE_NAME"][i])
    county_dict["state_FIPS"] = str(county_centroids_v2_df["STATEFP"][i])
    county_dict["properties"] = {}
    missing_array = []
    j = 0
    while j < len(missing_df):
        # check if county_fips matches
        if missing_df['County_FIPS'][j] == county_centroids_v2_df["GEOID"][i]:
            missing_dict = {}
            for item in missing_header:
                missing_dict[item] = str(missing_df[item][j])
            # append dictionary to missing array
            missing_array.append(missing_dict)
        # increment j
        j += 1
    county_dict["properties"]['missing'] = missing_array     
    
    unclaimed_array = []
    k = 0
    while k < len(unclaimed_df):
        # check if county_fips matches
        if unclaimed_df['County_FIPS'][k] == county_centroids_v2_df["GEOID"][i]:
            unclaimed_dict = {}
            for item in unclaimed_header:
                unclaimed_dict[item] = str(unclaimed_df[item][k])
            # append dictionary to unclaimed array
            unclaimed_array.append(unclaimed_dict)
        # increment k
        k += 1
    county_dict["properties"]['unclaimed'] = unclaimed_array
    
    unidentified_array = []
    l = 0
    while l < len(unidentified_df):
        # check if county_fips matches
        if unidentified_df['County_FIPS'][l] == county_centroids_v2_df["GEOID"][i]:
            unidentified_dict = {}
            for item in unidentified_header:
                unidentified_dict[item] = str(unidentified_df[item][l])
            # append dictionary to unclaimed array
            unidentified_array.append(unidentified_dict)
        # increment l
        l += 1
    county_dict["properties"]['unidentified'] = unidentified_array
    
    county_dict["properties"]['filtered'] = []
    ## set geometry
    county_dict["geometry"] = {}
    county_dict["geometry"]["type"] = "Point"
    county_dict["geometry"]["coordinates"] = [county_centroids_v2_df['Lon_dd'][i], county_centroids_v2_df['Lat_dd'][i]]
    ## append county dictionary to array
    county_array.append(county_dict)
    # test statement
    print("added county", str(i+1), "of 3283")
    # increment interator
    i += 1
# county_array

In [ ]:
# check item in array
len(county_array)
# county_array[55]['properties']['unclaimed']

In [ ]:
# Create FeatureCollection
county_feature_collection = {}
county_feature_collection["type"] = "FeatureCollection"
county_feature_collection["features"] = county_array
# county_feature_collection

In [ ]:
# Convert FeatureCollection to JSON format
county_geojson = geojson.dumps(county_feature_collection)
# check type to make sure conversion was sucessful
print(type(county_geojson))

In [ ]:
# Save JSON-formatted FeatureCollection as JSON file
# Save as new json file
with open('JSON/county_geojson.json', 'w', encoding='utf-8') as json_file:
    json_file.write(county_geojson)

### Construct CITY-LEVEL GeoJSON with correct structure

Goal format:
{
"type": "Feature",
    "name": "Dane",
    "state": "Wisconsin",
    "state_FIPS": ,
    "county_FIPS": ,
    "properties": {
        "missing": [ ],
        "unclaimed": [ ],
        "unidentified": [ ],
        "filtered": [ ]
    }
    "geometry": {
          "type": "Point",
          "coordinates": [
            -117.79750667,
            36.03755926
          ]
}
* each array will be a list of dictionaries. Each dictionary = one case. Keys = headers

In [ ]:
# get headers
missing_header = list(missing_df.columns.values)
print("missing header:", missing_header)
unclaimed_header = list(unclaimed_df.columns.values)
print("unclaimed header:", unclaimed_header)
unidentified_header = list(unidentified_df.columns.values)
print("unidentified header:", unidentified_header)

In [ ]:
# check citiess
city_df.head()
# len(city_df)

In [ ]:
# sort each database by county FIPS
# sort county centroids by county FIPS
city_df = city_df.sort_values(by=['COUNTYFP_CITY'])
city_df.head()

In [ ]:
# sort each database by county_FIPS_city
# sort missing by county FIPS
missing_df = missing_df.sort_values(by=['County_city'])
missing_df.head()

In [ ]:
# sort unclaimed by county FIPS
unclaimed_df = unclaimed_df.sort_values(by=['County_city'])
unclaimed_df.head()

In [ ]:
# sort unidentified by county FIPS
unidentified_df = unidentified_df.sort_values(by=['County_city'])
unidentified_df.head()

In [ ]:
city_df.head()

In [ ]:
len(city_df)

In [ ]:
## Create county array 
city_array = []
## for each county in county_centroids_v2... len(city_df)
i = 0
while i < 25:
    city_dict = {}
    city_dict["type"] = "Feature"
    city_dict["name"] = str(city_df["CITY_NAME"][i])
    city_dict["city_code"] = str(city_df["COUNTYFP_CITY"][i])
    city_dict["county_name"] = str(city_df["COUNTY_NAME"][i])
    city_dict["county_FIPS"] = str(city_df["COUNTY_FIPS"][i])
    city_dict["state_name"] = str(city_df["STATE_NAME"][i])
    city_dict["state_abbr"] = str(city_df["STUSPS"][i])
    city_dict["state_FIPS"] = str(city_df["STATEFP"][i])
    
    city_dict["properties"] = {}
    missing_array = []
    j = 0
    while j < len(missing_df):
        # check if county_city code matches
        if missing_df['County_city'][j] == city_df["COUNTYFP_CITY"][i]:
            missing_dict = {}
            for item in missing_header:
                missing_dict[item] = str(missing_df[item][j])
            # append dictionary to missing array
            missing_array.append(missing_dict)
        # increment j
        j += 1
    city_dict["properties"]['missing'] = missing_array     
    
    unclaimed_array = []
    k = 0
    while k < len(unclaimed_df):
        # check if county_city code matches
        if unclaimed_df['County_city'][k] == city_df["COUNTYFP_CITY"][i]:
            unclaimed_dict = {}
            for item in unclaimed_header:
                unclaimed_dict[item] = str(unclaimed_df[item][k])
            # append dictionary to unclaimed array
            unclaimed_array.append(unclaimed_dict)
        # increment k
        k += 1
    city_dict["properties"]['unclaimed'] = unclaimed_array
    
    unidentified_array = []
    l = 0
    while l < len(unidentified_df):
        # check if county_city code matches
        if unidentified_df['County_city'][l] == city_df["COUNTYFP_CITY"][i]:
            unidentified_dict = {}
            for item in unidentified_header:
                unidentified_dict[item] = str(unidentified_df[item][l])
            # append dictionary to unclaimed array
            unidentified_array.append(unidentified_dict)
        # increment l
        l += 1
    city_dict["properties"]['unidentified'] = unidentified_array
    
    city_dict["properties"]['filtered'] = []
    ## set geometry
    city_dict["geometry"] = {}
    city_dict["geometry"]["type"] = "Point"
    city_dict["geometry"]["coordinates"] = [city_df['Lon_dd'][i], city_df['Lat_dd'][i]]
    ## append county dictionary to array
    city_array.append(city_dict)
    # test statement
    print("added city", str(i+1), "of 199572")
    # increment interator
    i += 1
# city_array[0]

In [ ]:
# check item in array
len(city_array)
# city_array[55]['properties']['unclaimed']

In [ ]:
# Create FeatureCollection
city_feature_collection = {}
city_feature_collection["type"] = "FeatureCollection"
city_feature_collection["features"] = city_array
# city_feature_collection

In [ ]:
# Convert FeatureCollection to JSON format
city_geojson = geojson.dumps(city_feature_collection)
# check type to make sure conversion was sucessful
print(type(city_geojson))

In [ ]:
# Save JSON-formatted FeatureCollection as JSON file
# Save as new json file
with open('JSON/city_geojson.json', 'w', encoding='utf-8') as json_file:
    json_file.write(city_geojson)

# PART 6: State-level summary data (count for all 3 databases, by state)

### 1/4: Get count of missing person cases

In [ ]:
missing_count = missing_df.groupby('State').count()
len(missing_count)
missing_count.head()

In [ ]:
# Remove all columns except case number count
missing_count = missing_count.drop(columns=['DLC','Last Name', 'First Name', 'Missing Age', 'City', 'County', 'Sex', 'Race / Ethnicity', 'Date Modified', 'State_FIPS', 'State_County', 'County_FIPS'])

In [ ]:
# add column for state (since state is now index)
missing_count['State'] = missing_count.index

In [ ]:
# rename case number count column
missing_count = missing_count.rename(columns = {'Case Number': 'Missing_CaseCount'}, inplace = False)

In [ ]:
# check dataframe
missing_count.head()

In [ ]:
# make dictionary of states and missing person counts
missing_dict = dict(zip(missing_count.State, missing_count.Missing_CaseCount))
len(missing_dict)
# missing_dict

In [ ]:
# Get count of values in missing database w/ no state assigned
mis_null_series = missing_df.loc[missing_df['State']=='None'].count()
mis_null_ct = mis_null_series['Case Number']
mis_null_ct

### 2/4: Get count of unclaimed persons

In [ ]:
unclaimed_count = unclaimed_df.groupby('State').count()
# len(unclaimed_count)
unclaimed_count.head()

In [ ]:
# Remove all columns except case number count
unclaimed_count = unclaimed_count.drop(columns=['DBF','Last Name', 'First Name', 'Sex', 'Race / Ethnicity', 'City', 'County', 'Date Modified', 'State_FIPS', 'State_County', 'County_FIPS'])

In [ ]:
# add column for state (since state is now index)
unclaimed_count['State'] = unclaimed_count.index

In [ ]:
# rename case number count column
unclaimed_count = unclaimed_count.rename(columns = {'Case Number': 'Unclaimed_CaseCount'}, inplace = False)

In [ ]:
# check dataframe
unclaimed_count.head()

In [ ]:
# make dictionary of states and unclaimed case counts
unclaimed_dict = dict(zip(unclaimed_count.State, unclaimed_count.Unclaimed_CaseCount))
len(unclaimed_dict)
# unclaimed_dict

In [ ]:
# Get count of values in unclaimed database w/ no state assigned
unc_null_series = unclaimed_df.loc[unclaimed_df['State']=='None'].count()
unc_null_ct = unc_null_series['Case Number']
unc_null_ct

### 3/4: Get count of unidentified persons

In [ ]:
unidentified_count = unidentified_df.groupby('State').count()
# len(unidentified_count)
unidentified_count.head()

In [ ]:
# Remove all columns except case number count
unidentified_count = unidentified_count.drop(columns=['DBF','Age From', 'Age To', 'City', 'County', 'Sex', 'Race / Ethnicity', 'Date Modified', 'State_FIPS', 'State_County', 'County_FIPS'])

In [ ]:
# add column for state (since state is now index)
unidentified_count['State'] = unidentified_count.index

In [ ]:
# rename case number count column
unidentified_count = unidentified_count.rename(columns = {'Case Number': 'Unidentified_CaseCount'}, inplace = False)

In [ ]:
# check dataframe
unidentified_count.head()

In [ ]:
# make dictionary of states and unidentified case counts
unidentified_dict = dict(zip(unidentified_count.State, unidentified_count.Unidentified_CaseCount))
len(unidentified_dict)
# unidentified_dict

In [ ]:
# Get count of values in unidentified database w/ no state assigned
uni_null_series = unidentified_df.loc[unidentified_df['State']=='None'].count()
uni_null_ct = uni_null_series['Case Number']
uni_null_ct

### 4/4: Make summary dataframe

In [ ]:
# make a new summary dataframe based on the state centroids
summary_df = state_centroids_v2_df
# sort by state name
summary_df = summary_df.sort_values(by=['STATEFP'])
summary_df.head()

In [ ]:
# Remove all columns except state info and case number counts
summary_df = summary_df.drop(columns=['OBJECTID','STATENS','AFFGEOID','GEOID','LSAD','ALAND','AWATER','ORIG_FID'])
summary_df.head()

In [ ]:
len(summary_df)

#### 4a: Add Missing Person count for each state

In [ ]:
# Add new column to dataframe, using the state name field as a key in the missing_dict, to pull the correct missing case count for each state
summary_df['Missing_Count'] = summary_df['NAME'].map(missing_dict)
summary_df.head()

In [ ]:
# Check null values [NOTE: seems fine to have nulls]
missing_null_df = summary_df.loc[summary_df['Missing_Count'].isnull()]
missing_null_df

#### 4b: Add Unclaimed Person count for each state

In [ ]:
# Add new column to dataframe, using the state name field as a key in the unclaimed_dict, to pull the correct unclaimed case count for each state
summary_df['Unclaimed_Count'] = summary_df['NAME'].map(unclaimed_dict)
summary_df.head()

In [ ]:
# Check null values [NOTE: seems fine to have nulls]
unclaimed_null_df = summary_df.loc[summary_df['Unclaimed_Count'].isnull()]
unclaimed_null_df

In [ ]:
# No longer needed, since summary dataframe now made AFTER data cleaning and adding State with FIPS 99
# # # change NaN Unclaimed count for fake FIPS 99 to count of values in missing database w/ no state assigned (21)
# index_Series = summary_df.loc[summary_df['STATEFP']==99]
# index_None = index_Series.index[0]
# summary_df.loc[index_None, 'Unclaimed_Count'] = unc_null_ct
# # check value
# summary_df['Unclaimed_Count'][index_None]

#### 4b: Add Unidentified Person count for each state

In [ ]:
# Add new column to dataframe, using the state name field as a key in the unidentified_dict, to pull the correct unidentified case count for each state
summary_df['Unidentified_Count'] = summary_df['NAME'].map(unidentified_dict)
summary_df.head()

In [ ]:
# Check null values [NOTE: seems fine to have nulls]
unidentified_null_df = summary_df.loc[summary_df['Unidentified_Count'].isnull()]
unidentified_null_df

#### 4c: Add column for total count of cases in all three databases for each state

In [ ]:
summary_df.head()

In [ ]:
# Note - may need to change -4 to -3, run, then change back to -4 and re-run. DON'T KNOW WHY
summary_df['Total_Count'] = summary_df.iloc[:, -4:-1].sum(axis=1)
summary_df.head(56)

#### 4c: Convert to geodataframe and export as GeoJSON

In [ ]:
# check final summary_df
summary_df.head()

In [ ]:
# Convert dataframe to geodataframe
summary_gdf = gpd.GeoDataFrame(summary_df, geometry=gpd.points_from_xy(x=summary_df.Lon_dd, y=summary_df.Lat_dd))
summary_gdf.head()

In [ ]:
# write to geoJSON
summary_gdf.to_file("JSON/summary_counts.json", driver="GeoJSON", encoding='utf-8')

# PART 7: County-level summary data (count for all 3 databases, by state)

### 1/4: Get count of missing person cases at county level

In [ ]:
missing_df.head()

In [ ]:
missing_county_count = missing_df.groupby('County_FIPS').count()
# len(missing_county_count)
missing_county_count.head()

In [ ]:
# Remove all columns except case number count
missing_county_count = missing_county_count.drop(columns=['DLC','Last Name', 'First Name', 'Missing Age', 'City', 'County', 'State', 'Sex', 'Race / Ethnicity', 'Date Modified', 'State_FIPS', 'State_County'])

In [ ]:
# add column for state (since state is now index)
missing_county_count['County_FIPS'] = missing_county_count.index

In [ ]:
# rename case number count column
missing_county_count = missing_county_count.rename(columns = {'Case Number': 'Missing_CaseCount'}, inplace = False)

In [ ]:
# check dataframe
missing_county_count.head()

In [ ]:
# make dictionary of county_FIPS and missing person counts
missing_county_dict = dict(zip(missing_county_count.County_FIPS, missing_county_count.Missing_CaseCount))
len(missing_county_dict)
# missing_county_dict

### 2/4: Get count of unclaimed person cases at county level

In [ ]:
unclaimed_df.head()

In [ ]:
unclaimed_county_count = unclaimed_df.groupby('County_FIPS').count()
# len(unclaimed_county_count)
unclaimed_county_count.head()

In [ ]:
# Remove all columns except case number count
unclaimed_county_count = unclaimed_county_count.drop(columns=['DBF','Last Name', 'First Name', 'Sex', 'Race / Ethnicity', 'City', 'County', 'State', 'Date Modified', 'State_FIPS', 'State_County'])

In [ ]:
# add column for state (since state is now index)
unclaimed_county_count['County_FIPS'] = unclaimed_county_count.index

In [ ]:
# rename case number count column
unclaimed_county_count = unclaimed_county_count.rename(columns = {'Case Number': 'Unclaimed_CaseCount'}, inplace = False)

In [ ]:
# check dataframe
unclaimed_county_count.head()

In [ ]:
# make dictionary of county_FIPS and missing person counts
unclaimed_county_dict = dict(zip(unclaimed_county_count.County_FIPS, unclaimed_county_count.Unclaimed_CaseCount))
len(unclaimed_county_dict)
# unclaimed_county_dict

### 3/4: Get count of unidentified person cases at county level

In [ ]:
unidentified_df.head()

In [ ]:
unidentified_county_count = unidentified_df.groupby('County_FIPS').count()
# len(unidentified_county_count)
unidentified_county_count.head()

In [ ]:
# Remove all columns except case number count
unidentified_county_count = unidentified_county_count.drop(columns=['DBF','Age From', 'Age To', 'City', 'County', 'State', 'Sex', 'Race / Ethnicity', 'Date Modified', 'State_FIPS', 'State_County'])

In [ ]:
# add column for state (since state is now index)
unidentified_county_count['County_FIPS'] = unidentified_county_count.index

In [ ]:
# rename case number count column
unidentified_county_count = unidentified_county_count.rename(columns = {'Case Number': 'Unidentified_CaseCount'}, inplace = False)

In [ ]:
# check dataframe
unidentified_county_count.head()

In [ ]:
# make dictionary of county_FIPS and missing person counts
unidentified_county_dict = dict(zip(unidentified_county_count.County_FIPS, unidentified_county_count.Unidentified_CaseCount))
len(unidentified_county_dict)
# unidentified_county_dict

### 4/4: Make summary county-level dataframe

In [ ]:
county_centroids_v2_df.head()

In [ ]:
# make a new summary dataframe based on the state centroids
summary_county_df = county_centroids_v2_df
# sort by county FIPS
summary_county_df = summary_county_df.sort_values(by=['GEOID'])
summary_county_df.head()

In [ ]:
# Remove all columns except county info and case number counts
summary_county_df = summary_county_df.drop(columns=['OBJECTID','COUNTYFP','COUNTYNS','AFFGEOID','LSAD','ALAND','AWATER','ORIG_FID'])
summary_county_df.head()

In [ ]:
len(summary_county_df)

#### 4a: Add Missing Person count for each county

In [ ]:
# Add new column to dataframe, using the county FIPS field as a key in the missing_county_dict, to pull the correct missing case count for each county
summary_county_df['Missing_Count'] = summary_county_df['GEOID'].map(missing_county_dict)
summary_county_df.head()

In [ ]:
# Check null values [NOTE: fine to have nulls]
missing_county_null_df = summary_county_df.loc[summary_county_df['Missing_Count'].isnull()]
missing_county_null_df

#### 4b: Add Unclaimed Person count for each county

In [ ]:
# Add new column to dataframe, using the county FIPS field as a key in the unclaimed_county_dict, to pull the correct unclaimed case count for each county
summary_county_df['Unclaimed_Count'] = summary_county_df['GEOID'].map(unclaimed_county_dict)
summary_county_df.head()

In [ ]:
# Check null values [NOTE: fine to have nulls]
unclaimed_county_null_df = summary_county_df.loc[summary_county_df['Unclaimed_Count'].isnull()]
unclaimed_county_null_df

#### 4b: Add Unidentified Person count for each county

In [ ]:
# Add new column to dataframe, using the county_FIPS field as a key in the unidentified_county_dict, to pull the correct unidentified case count for each county
summary_county_df['Unidentified_Count'] = summary_county_df['GEOID'].map(unidentified_county_dict)
summary_county_df.head()

In [ ]:
# Check null values [NOTE: fine to have nulls]
unidentified_county_null_df = summary_county_df.loc[summary_county_df['Unidentified_Count'].isnull()]
unidentified_county_null_df

#### 4c: Add column for total count of cases in all three databases for each county

In [ ]:
summary_county_df.head()

In [ ]:
# Note - may need to change -4 to -3, run, then change back to -4 and re-run. DON'T KNOW WHY
summary_county_df['Total_Count'] = summary_county_df.iloc[:, -4:-1].sum(axis=1)
summary_county_df.head()

#### 4c: Convert to geodataframe and export as GeoJSON

In [ ]:
# check final summary_county_df
summary_county_df.head()

In [ ]:
len(summary_county_df)

In [ ]:
# Convert dataframe to geodataframe
summary_county_gdf = gpd.GeoDataFrame(summary_county_df, geometry=gpd.points_from_xy(x=summary_county_df.Lon_dd, y=summary_county_df.Lat_dd))
summary_county_gdf.head()

In [ ]:
len(summary_county_gdf)

In [ ]:
# write to geoJSON
summary_county_gdf.to_file("JSON/county_counts.json", driver="GeoJSON", encoding='utf-8')

# Part 8 make State and County polygon GeoJSONS

In [ ]:
# Load state shapefile
state_gdf = gpd.read_file('Shapefiles/state_polygons.shp')

In [ ]:
state_gdf.plot()

In [ ]:
# write to geoJSON
state_gdf.to_file("JSON/state_polygons.json", driver="GeoJSON", encoding='utf-8')

In [ ]:
# load county shapefile
county_gdf = gpd.read_file('Shapefiles/county_polygons.shp')

In [ ]:
county_gdf.plot()

In [ ]:
# write to geoJSON
county_gdf.to_file("JSON/county_polygons.json", driver="GeoJSON", encoding='utf-8')